# Technical demand response potential meta analysis - result comparison (overview)

**Puporse and background**: This notebook serves for evaluating the demand response potentials from different publications assessing technical demand response potentials for Germany. The data from the publications has been collected in order to carry out a meta analysis.

## Method applied
In the following, a brief description of the method applied is given for the sake of replicability.

### Selection of publications
Criterions for the selection of publications were:
* Regional focus: Federal republic of Germany
* Temporal restricitions: Publications until 2005
* Contents: technical demand response potentials
* Sectoral coverage: At least one of households, trade, commerce and services or industry assessed; no publications focusing on single appliances
* Own analysis carried out (no other meta analysis included)
* Inclusion of at least one of the parameters of interest (usually at least some information on potential in MW)

### Parameters assessed
Data for the following parameters was taken from the publications selected:
* eligibility: general eligibility of processes, applications or branches (demand response categories in the following) for load shifting, load shedding resp. (sole) load increase
* capacity-related parameters: capacity available for load shifting or load shedding, average, minimum and maximum load level, installed capacity as well as shiftable share of installed capacity
* time-reletad parameters: activation duration, interference duration, shifting duration, regenaration duration, maximum activations per year
* cost-related parameters: specific investments, variable costs (for activation), fixed costs

### Filtering and data collection
* A total of 70 publications has been selected in the first place in a literature research based on the titles, keywords and abstracts using terms like "Demand Response", "Demand Side Management", "Demand Side Integration", "load management", "load shifting", "load shedding", "potential", "Germany" as well as their German equivalents to formulate search strings. 
* Of these publications, 30 have met the above stated criteria and were further evaluated. 
* Because sometimes, several publications belonged to the same research project, these were grouped together resulting in 24 publication groups for which potential information has been evaluated.
* The data has been collected in an Excel Workbook ("Potenziale_Lastmanagement.xlsx"). The latest version of this Excel Workbook is evaluated with this notebook.
***
> _NOTE:_ 
> * _This notebook offers some room for improvement in terms of computing and coding efficiency._
> * _Pandas setting a value with a copy warning has been disabled. Instead, function definitions should be revised such that this is not necessary._

# Package imports
User-defined functions are imported in order to prevent redundancies:
* *extract_info*: extracts the information for the relevant parameter out of the complete input data
* *create_info_dict_for_years*: creates a dictionary for the relevant parameter info indexed by years
* *save_info_stats*: writes some info statistics into an Excel workbook
* *save_sources*: writes the sources information into an Excel workbook
* *get_nlargest*: determines the demand response categories with the largest potential
* *create_info_boxplot*: creates a box plot to visualize the parameter information
* *extract_stats*: determines some statistical measures of central tendency for a demand response paramter
* *create_projection_boxplot*: creates a box plot for future projections
* *save_future_sources*: writes the sources for future projections into an Excel workbook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlrd

from potential_evaluation_funcs import (extract_info, create_info_dict_for_years, 
                                        save_info_stats, save_sources, create_info_boxplot,
                                        get_nlargest, extract_stats, create_projection_boxplot, 
                                        save_future_sources)

# Allow for chained assignments (see note above)
pd.options.mode.chained_assignment = None

# Parameter settings
Set some (mostly boolean) parameters here in order to ensure code adaptability to future versions of potential collection (Excel workbook).
The following parameters determine the general workflow:

| Parameter | Purpose |
| :-: | :- |
| *write_outputs* | If True, statistical information for demand response parameters is written to Excel files. |
| *write_sources* | If True, sources information is written to Excel files. |
| *presentation_stuff* | If True, some rearrangement of column order is done in order to create nicer box plot<br> visualizations which can be used for presentation slides (only for certain demand response parameters). |

Some of the parameters are used to control user-defined function behaviour. These are set centrally here, but may be adapted for individual function calls. The following boolean parameters control function behaviour:

| Parameter | Purpose | Comment |
| :-: | :- | :- |
| *drop* | If True, all processes for which no information on a certain parameter is available will not be shown in the boxplots |
| *drop_datalack* | If True, all processes with less than 3 entries for a given parameter will be excluded. |
| *sample_stats* | If True, info on sample size will be calculated based on the data and shown in boxplots. | _Warning: activation leads to relatively high computational effort._ |
| *add_sources* | If True, source information for the data is collected. | _Warning: activation leads to relatively high computational effort._ |
| *modify_categories* | If True, the original demand response categories from the studies are replaced with self-defined demand response categories in order to aggregate them. There is the possibility to choose, how granular the aggregation will be. | _Warning: activation leads to relatively high computational effort._ |
| *aggregation_mode* | - Defines, how strongly the data will be aggregated if modify_categories is set to True.<br>- A broader aggregation increases the sample sizes but is more inaccurate since the underlying assumption in the grouping done here is that the profiles are identical.<br>- Options: "granular" which performs a decent aggregation vs. "broad" which combines lots of processes and applications into one category. |

The following parameters control plot ouputs:

| Paraneter | Purpose | Comment |
| :-: | :- | :- |
| *use_limits* | If True, the limits for the boxplot yaxis values are obtained from the data itself<br>(overall minimum / maximum in DataFrame to plot). | _Note: If parameter is set to True, yaxis limits given by the function<br>parameter ylim will be ignored._ |
| *use_colors* | If True, color will be chosen for a given process. |
| *show_title* | If True, the title for boxplots will be displayed. |
| *swarmplot* | If True, a bee swarm plot will be plotted on top of the boxplots<br>(using seaborn for plotting). |
| *savefig* | If True, boxplots are saved to .png files. |

The following parameters control process selection:

| Parameter | Purporse |
| :-: | :- |
| metric | Determines which metric shall be used to select processes for further analysis (median, mean, etc.) |
| n | Determines the n largest values for a certain metric to be further analyzed. |
| use_nlargest_cols | - If True, the processes resp. applications with the highest potentials are evaluated in the potential outlook.<br>- If False, a set of processes resp. applications is chosen that has been self-defined (instead of changing every single function parameter).<br>- If further options shall be incorporated, replace with a dict holding these options instead. |

> _NOTE: For usage within the clustering notebooks, the following parameter settings are needed to create the correct parameter statistics outputs which are read in in the clustering process:_
> * *write_outputs=True*
> * *drop=True*
> * *drop_datalack=True*
> * *modify_categories=True*
> * *aggregation_mode='granular'*

The remainder of the parameters is only relevant for controlling the plots and sources information in this notebook and can be freely chosen.

In [ ]:
# Set paths and filenames for in- and outputs
path_folder_in = "./"
path_folder_out = "./out/"
path_folder_stats = path_folder_out + "stats/"
path_folder_sources = path_folder_out + "sources/"
path_folder_plots = path_folder_out + "plots/"

filename = "Potenziale_Lastmanagement.xlsx"
# If no sources are added, columns containing information on sources are ignored
filename_colnames = "Columns.xlsx"

# boolean parameters for controlling the workflow
write_outputs = True
write_sources = False
presentation_stuff = False

# boolean parameters for controlling function behaviour
drop = True
drop_datalack = True
sample_stats = True
add_sources = False
# Determine aggregation_mode if modify_categories is set to True
modify_categories = True
# Alternatives for aggregation mode: "granular" vs. "broad"
aggregation_mode = "granular" 

# boolean parameters for controlling plot outputs
use_limits = True
use_colors = True          
show_title = False
swarmplot = True
savefig = False

# parameters for process selection for further analysis
metric = "50%"
n = 5
use_nlargest_cols = False

Configure the fontsizes for matplotlib objects

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Read in data
* Read in the Excel Workbook containing all data
* create a separate DataFrame with the potential information for every publication (which will be sliced and combined with the respective potential information from other studies in order to evaluate the respective parameter / appliance).

In [ ]:
# Get all sheet names using xlrd
xls = xlrd.open_workbook(path_folder_in+filename, on_demand=True)
table_names = xls.sheet_names()

# Remove all sheets that do not contain potential information as well as worksheets from publications which
# were excluded in the process and include sheets "Dropdown" and "Kategorien"
removed_names = {el for el in table_names if len(el) > 6}

# Two publications were excluded in the latter process
# The worksheets were not removed but hided, hence, have to be removed here
removed_names.add("Stoe12")
removed_names.add("Bel15")
table_names = [el for el in table_names if el not in removed_names]

# The worksheets dropdown (containing all feasible dropdown options), categories (containing processes) 
# and colors (containing a color mapping for processes) shall be evaluated, too
table_names.append("Dropdown")
table_names.append("Kategorien")
table_names.append("Farben")
print(table_names)

In [ ]:
# Read in data using pandas and parse it to individual DataFrames for each publication
xls = pd.ExcelFile(path_folder_in+filename)

ape12 = xls.parse("Ape12")
ary17 = xls.parse("Ary17")
blu13 = xls.parse("Blu13")
foc11 = xls.parse("Foc11")
gil15 = xls.parse("Gil15")
gob12 = xls.parse("Gob12")
gro13 = xls.parse("Gro13")
gru17 = xls.parse("Gru17")
haa17 = xls.parse("Haa17")
hen15 = xls.parse("Hen15")
klo09 = xls.parse("Klo09")
klo13 = xls.parse("Klo13")
krz13 = xls.parse("Krz13")
lad18 = xls.parse("Lad18")
lan15 = xls.parse("Lan15")
lie15 = xls.parse("Lie15")
mol10 = xls.parse("Mol10")
pau11 = xls.parse("Pau11")
pel16 = xls.parse("Pel16")
r2b14 = xls.parse("r2b14")
roo10 = xls.parse("Roo10")
sta06 = xls.parse("Sta06")
ste17 = xls.parse("Ste17")
sty15 = xls.parse("Sty15")

# Read in dropdown options, categories as well as colors used, as well
dropdown = xls.parse("Dropdown")
if aggregation_mode == "granular":
    categories = xls.parse("Kategorien")
elif aggregation_mode == "broad":
    categories = xls.parse("Kategorien_grob")
colors = xls.parse("Farben", index_col=0, header=0)

# Prepare data for further analysis
In the following, some data preparation steps are carried out in order to harmonize the data basis and ensure further usability.

## General routines
The following general routines introduce dicts to easily adress all sheets / DataFrames using for loops. Furthermore, the years considered are grouped in order to distinct status quo considerations from projections for the future. Furthermore, the data basis is split up by sectors and years since this is often needed in the following.

### Publications dict (shortcut)
Introduce a dict matching the DataFrames to a string name for quicker handling

In [ ]:
publications_dict = {"Ape12": ape12,
                     "Ary17": ary17,
                     "Blu13": blu13,
                     "Foc11": foc11,
                     "Gil15": gil15,
                     "Gob12": gob12,
                     "Gro13": gro13,
                     "Gru17": gru17,
                     "Haa17": haa17,
                     "Hen15": hen15,
                     "Klo09": klo09,
                     "Klo13": klo13,
                     "Krz13": krz13,
                     "Lad18": lad18,
                     "Lan15": lan15,
                     "Lie15": lie15,
                     "Mol10": mol10,
                     "Pau11": pau11,
                     "Pel16": pel16,
                     "r2b14": r2b14,
                     "Roo10": roo10,
                     "Sta06": sta06,
                     "Ste17": ste17,
                     "Sty15": sty15}

In [ ]:
ape12

### Years
Evaluate the years for which potentials are given in order to group them into the following groups:
* Status quo: prior to 2020
* 2020
* 2025 (including 2023)
* 2030
* 2035
* 2040
* 2045
* 2050

In [ ]:
# Get the years for which potentials are given
set_years = set()

for k, v in publications_dict.items():
    for el in v["Jahr"].unique():
        set_years.add(el)

# Define two groups of years (for potential evaluation)
status_quo_years = {el for el in set_years if el < 2020}
years_2025 = {2023, 2025}

# Define year dict for checking the year (group)
years_dict = {"SQ": status_quo_years,
              "2020": [2020],
              "2025": years_2025,
              "2030": [2030],
              "2035": [2035],
              "2040": [2040],
              "2045": [2045],
              "2050": [2050]}

### Processes
Obtain sets of processes from Excel workbook:
* processes_original: Original listing of processes
* processes_categorized: Categorization of processes (same as in method comparison)

In [ ]:
# Original set of processes
processes_original = set(dropdown["Prozess"].values)

# Categorized set of processes
cat1 = categories["Prozesskategorie 1"].values
cat2 = categories["Prozesskategorie 2"].dropna().values
cat3 = categories["Prozesskategorie 3"].dropna().values
processes_categorized = set(np.concatenate((cat1, cat2, cat3), axis = 0))

# Compare the number of processes before and after adjusting the categorization
print(len(processes_original), len(processes_categorized))

In [ ]:
categories.head()

### Publications per sector and year (dict as shortcut)
* Create DataFrames (subsets of overall data tables) which focus on a certain sector and year and store them in a dict indexed by publications and years resp. year groups
* Include the adjusted process categorization in order to be able to use this for evaluation as well
    * This operation is more or less equivalent to assigning the process categorization in the original data set which could be achieved using the Excel function VLOOKUP (SVERWEIS in German).
    * It is implemented by merging the two DataFrames to a combined one since the indices differ and thus cannot be used for comparison.

In [ ]:
publications_ind_years_dict = {}
publications_tcs_years_dict = {}
publications_hoho_years_dict = {}

for year_key, years in years_dict.items():
    for k, v in publications_dict.items():
        # Extract data for the industry sector
        publications_ind_years_dict[k+"_"+year_key] = v[(v["Sektorenzuordnung"] == "Industrie") & (v["Jahr"].isin(years))]
        publications_ind_years_dict[k+"_"+year_key]["year_key"] = year_key
        publications_ind_years_dict[k+"_"+year_key] = \
            pd.merge(publications_ind_years_dict[k+"_"+year_key], categories, left_on="Prozess", right_on="Prozess (ursprüngliche Kategorien)", how="inner").drop("Prozess (ursprüngliche Kategorien)", axis=1)
        
        # Extract data for the trade, commerce and services sector
        publications_tcs_years_dict[k+"_"+year_key] = v[(v["Sektorenzuordnung"] == "GHD") & (v["Jahr"].isin(years))]
        publications_tcs_years_dict[k+"_"+year_key]["year_key"] = year_key
        publications_tcs_years_dict[k+"_"+year_key] = \
            pd.merge(publications_tcs_years_dict[k+"_"+year_key], categories, left_on="Prozess", right_on="Prozess (ursprüngliche Kategorien)", how="inner").drop("Prozess (ursprüngliche Kategorien)", axis=1)
        
        # Extract data for the household sector
        publications_hoho_years_dict[k+"_"+year_key] = v[(v["Sektorenzuordnung"] == "Haushalte") & (v["Jahr"].isin(years))]
        publications_hoho_years_dict[k+"_"+year_key]["year_key"] = year_key
        publications_hoho_years_dict[k+"_"+year_key] = \
            pd.merge(publications_hoho_years_dict[k+"_"+year_key], categories, left_on="Prozess", right_on="Prozess (ursprüngliche Kategorien)", how="inner").drop("Prozess (ursprüngliche Kategorien)", axis=1)

### Prepare demand response categories for further analysis
Due to the heterogenous demand response categories used, a harmonization / an own categorization is needed.
This is done by a mapping of original process resp. demand response category names to self-chosen ones (derived from the categories of the meta-analysis and mostly on a basis of processes / applications). 

Some original categories have to be grouped together. If this is the case, their availability profiles are assumed to be (more or less) identical. Then, aggregation rules are applied to the demand response parameters (sum resp. (weigthed) average).

For the decision, whether to just replace the original demand response category name or whether to apply aggregation rules, a variable *replacement_mode* is introduced:
* -1: Keep original column name since there is no equivalent alternative
* 0: Take column name from "Prozesskategorie 1" -> no duplicates (simple renaming)
* 1: Replace multiple columns with one value of "Prozesskategorie 1" -> duplicates (aggregation is carried out)

The replacement_mode is set here based on a prior visual inspection of the used categories (uncomment lines at the bottom to see the output).

In [ ]:
# Industry sector

if aggregation_mode == "granular":
    # dict holding the entries to be changed to 1
    replacement_dict = {"Blu13_SQ": [5, 6],
                        "Gil15_SQ": [2, 3],
                        "Klo09_SQ": [3, 4],
                        "Lad18_SQ": [2, 3],
                        "Gil15_2020": [2, 3],
                        "Klo09_2020": [2, 3],
                        "Gil15_2030": [2, 3],
                        "Gil15_2050": [2, 3]}

    # dict holding the entries to be changed to -1
    keep_dict = {"Lan15_SQ": [0, 4],
                 "r2b14_SQ": [1]}

elif aggregation_mode == "broad":
    # dict holding the entries to be changed to 1
    replacement_dict = {"Ape12_SQ": [2, 3],
                        "Blu13_SQ": [5, 6],
                        "Foc11_SQ": [2, 3, 4, 5],
                        "Gil15_SQ": [2, 3, 4, 5, 6],
                        "Gru17_SQ": [2, 3],
                        "Klo09_SQ": [3, 4, 5, 6, 7, 8],
                        "Klo13_SQ": [2, 3],
                        "Lad18_SQ": [2, 3],
                        "Mol10_SQ": [4, 8, 9],
                        "Pel16_SQ": [2, 3, 6, 10],
                        "Ste17_SQ": [9, 10, 11],
                        "Ape12_2020": [2, 3],
                        "Gil15_2020": [2, 3, 4, 5, 6],
                        "Gru17_2020": [2, 3],
                        "Klo09_2020": [2, 3, 4, 5, 6, 7],
                        "Pel16_2020": [2, 3],
                        "Ste17_2020": [9, 10, 11],
                        "Gru17_2025": [2, 3],
                        "Pel16_2025": [2, 3],
                        "Ste17_2025": [9, 10, 11],
                        "Ape12_2030": [2, 3],
                        "Gil15_2030": [2, 3, 4, 5, 6],
                        "Gru17_2030": [2, 3],
                        "Pel16_2030": [2, 3],
                        "Ste17_2030": [9, 10, 11],
                        "Ste17_2035": [9, 10, 11],
                        "Ste17_2040": [9, 10, 11],
                        "Ste17_2045": [9, 10, 11],
                        "Gil15_2050": [2, 3, 4, 5, 6],
                        "Ste17_2050": [9, 10, 11]}

    # dict holding the entries to be changed to -1
    keep_dict = {"Lan15_SQ": [0, 4],
                 "r2b14_SQ": [1]}

# Change the replacement_mode for the above stated entries
for k, v in publications_ind_years_dict.items():
    v["replacement_mode"] = 0
    if k in replacement_dict.keys():
        v.loc[replacement_dict[k], "replacement_mode"] = 1
    if k in keep_dict.keys():
        v.loc[keep_dict[k], "replacement_mode"] = -1
    
    #print(k)    
    #display(v.loc[:,["Prozess", "Prozesskategorie 1", "replacement_mode"]])

In [ ]:
# TCS sector

if aggregation_mode == "granular":
    # dict holding the entries to be changed to 1
    replacement_dict = {"Foc11_SQ": [0, 4],
                        "Gil15_SQ": [0, 1, 2],
                        "Klo09_SQ": [0, 1, 3, 4, 5, 6],
                        "Gil15_2020": [0, 1, 2],
                        "Klo09_2020": [0, 1, 3, 4, 5, 6],
                        "Gil15_2030": [0, 1, 2],
                        "Gil15_2050": [0, 1, 2]}

    # dict holding the entries to be changed to -1
    keep_dict = {}
    
elif aggregation_mode == "broad":
    # dict holding the entries to be changed to 1
    replacement_dict = {"Foc11_SQ": [0, 4],
                        "Gil15_SQ": [0, 1, 2],
                        "Klo09_SQ": [0, 1, 2, 3, 4, 5, 6],
                        "Gil15_2020": [0, 1, 2],
                        "Klo09_2020": [0, 1, 2, 3, 4, 5, 6],
                        "Gil15_2030": [0, 1, 2],
                        "Gil15_2050": [0, 1, 2]}

    # dict holding the entries to be changed to -1
    keep_dict = {}

# Change the replacement_mode for the above stated entries
for k, v in publications_tcs_years_dict.items():
    v["replacement_mode"] = 0
    if k in replacement_dict.keys():
        v.loc[replacement_dict[k], "replacement_mode"] = 1
    if k in keep_dict.keys():
        v.loc[keep_dict[k], "replacement_mode"] = -1   
    
    #print(k)    
    #display(v.loc[:,["Prozess", "Prozesskategorie 1", "replacement_mode"]])

In [ ]:
# Household sector

if aggregation_mode == "granular":
    # dict holding the entries to be changed to 1
    replacement_dict = {}

    # dict holding the entries to be changed to -1
    keep_dict = {"Gro13_SQ": [0],
                "Klo13_SQ": [2],
                "Mol10_SQ": [6],
                "Pel16_SQ": [4],
                "Sta06_SQ": [6],
                "Mol10_2020": [6],
                "Pel16_2020": [4],            
                "Pel16_2025": [4],            
                "Pel16_2030": [4]}

elif aggregation_mode == "broad":
    # dict holding the entries to be changed to 1
    replacement_dict = {"Ape12_SQ": [0, 7, 9],
                        "Ary17_SQ": [4, 5],
                        "Haa17_SQ": [0, 1, 2],
                        "Klo09_SQ": [3, 4],
                        "Mol10_SQ": [4, 5],
                        "Sta06_SQ": [1, 2],
                        "Ste17_SQ": [1, 4],
                        "Sty15_SQ": [0, 1],
                        "Ape12_2020": [0, 7, 9],
                        "Klo09_2020": [3, 4],
                        "Mol10_2020": [4, 5],
                        "Ste17_2020": [1, 4],
                        "Ste17_2025": [1, 4],
                        "Sty15_2025": [0, 1],
                        "Ape12_2030": [0, 7, 9],
                        "Ste17_2030": [1, 4],
                        "Ste17_2035": [1, 4],
                        "Ste17_2040": [1, 4],
                        "Ste17_2045": [1, 4],
                        "Ste17_2050": [1, 4],
                        "Sty15_2050": [0, 1]}

    # dict holding the entries to be changed to -1
    keep_dict = {"Gro13_SQ": [0],
                "Klo13_SQ": [2],
                "Mol10_SQ": [6],
                "Pel16_SQ": [4],
                "Sta06_SQ": [6],
                "Mol10_2020": [6],
                "Pel16_2020": [4],            
                "Pel16_2025": [4],            
                "Pel16_2030": [4]}

# Change the replacement_mode for the above stated entries
for k, v in publications_hoho_years_dict.items():
    v["replacement_mode"] = 0
    if k in replacement_dict.keys():
        v.loc[replacement_dict[k], "replacement_mode"] = 1
    if k in keep_dict.keys():
        v.loc[keep_dict[k], "replacement_mode"] = -1  
    
    #print(k)    
    #display(v.loc[:,["Prozess", "Prozesskategorie 1", "replacement_mode"]])

### Prepare colors
Include the values for "Prozesskategorie 1" from the categories DataFrame (and exclude duplicates).

In [ ]:
# Include values of "Prozesskategorie 1" from the categories DataFrame in the color mapping
new_colors = colors.copy()
new_colors["Prozesskategorie 1"] = categories.set_index("Prozess (ursprüngliche Kategorien)")["Prozesskategorie 1"]

new_colors = new_colors.set_index("Prozesskategorie 1")
new_colors = new_colors.loc[~new_colors.index.duplicated(keep="first")]
new_colors.index.name = "Prozess"

# concat original colors DataFrame with the one containing values for "Prozesskategorie 1"
# and remove duplicates
colors = pd.concat([colors, new_colors])
colors = colors.loc[~colors.index.duplicated(keep="first")]

# Show colors
colors

## Prepare column names
Column names have to be carefully selected and harmonized in order to be able to compare potential information on a common basis. This holds especially for evaluating capacity-related parameters.
* Get capacity-related columns:
    * Evaluate all columns holding the word "potential" or fractions of it.
    * Distinct between pos and negative potentials.
    * Remove columns evaluating sociotechnical instead of technical potentials as well as those evaluating energy instead of capacity.

Inspect columns by evaluating all column names used accross the publications

In [ ]:
# initialize an empty throwaway array
help_array = np.empty(shape=(0,))

# add all column values and convert to a set to remove duplicates
for k, v in publications_dict.items():
    help_array = np.concatenate((help_array, v.columns.values), axis = 0)
    
all_cols = set(help_array)

In [ ]:
# print the number of different column names used accross the publications
len(all_cols)

Prepare columns for further usage:
* The list of all columns used has been transferred to an Excel workbook using the following commented lines of code.
* The columns have been matched to one parameter for evaluation by going through them and assigning a parameter (has been done manually using Excel).
* The manipulated Excel workbook ("Columns.xlsx") is read in in the following again to be able to use the grouping applied.

In [ ]:
# Uncomment this cell and reassign groups if changes to the columns in the data collection occur.
#all_cols_Series = pd.Series(data=sorted(list(all_cols)))
#all_cols_Series.to_excel("Columns.xlsx", encoding="utf-8-sig", header=False)

In [ ]:
all_cols_df = pd.read_excel(path_folder_in+filename_colnames, index_col=0)

In [ ]:
# Assign the groups that have been defined to a variable to use it as key for a dict
groups = all_cols_df["Gruppe"].unique()
groups

In [ ]:
cols_dict = {}

for key in groups:
    cols_dict[key] = list(all_cols_df[all_cols_df["Gruppe"] == key].index.values)
    cols_dict[key].extend(["Prozesskategorie", "Jahr", "Sektorenzuordnung", "study"])

In [ ]:
cols_dict

# EXPERIMENTAL (IGNORE FOR NOW)
Experimental section:
* Use simple DataFrame filtering instead of creating extensive dict structures
* TODO: Resume when time permits / Remove if not!

**TODO: Fix data type error!**

Logic:
* Filter cols that don't need renaming
* Append the string for the study to all other cols
* concat to form one overall DataFrame
* Add study string to Fundstelle'

In [ ]:
cols_not_to_rename = [
    "Prozess", "Sektorenzuordnung", "Jahr", 
    "Lastverschiebung", "Lastverzicht", "Lasterhöhung", 
    "study"
]
cols_not_to_rename.extend([col for col in all_cols if "Fundstelle" in col])

In [ ]:
parameters = pd.DataFrame()
for k, v in publications_dict.items():
    v["study"] = k
    v.rename(columns={
        col: col + "_" + k for col in v.columns if col not in cols_not_to_rename
    }, inplace=True)
    parameters = pd.concat([parameters, v])

for el in parameters.columns:
    if "Fundstelle" in el:
        parameters[el] = parameters["study"].values + ": " + parameters[el].astype(str).values

In [ ]:
cats_new = xls.parse("Kategorien_neu")
cats_new

In [ ]:
parameters

Logic:
* Merge with new category mapping
* Define aggregation rules to groupby "Prozesskategorie"
* Define new aggregation dict taking into account the endings for the study values

In [ ]:
parameters = pd.merge(
    parameters, cats_new,
    left_on="Prozess", 
    right_on="Prozess (ursprüngliche Kategorien)",
    how="left"
)

In [ ]:
# Define sector filter
filter_sector = {
    "ind": "Industrie",
    "tcs": "GHD",
    "hoho": "Haushalte"
}

In [ ]:
agg_dict = all_cols_df["Aggregationsregel"].loc[
    all_cols_df["Aggregationsregel"] != "drop"
].to_dict()
agg_dict["study"] = "first"

In [ ]:
new_agg_dict = {}
for col in parameters.columns:
    for k, v in agg_dict.items():
        if k in col:
            new_agg_dict[col] = v

Logic:
* Iterate over studies, sectors and years
* Filter DataFrame accordingly and groupby "Prozesskategorie" to remove duplicates
_NOTE: Does not yet work!_

In [ ]:
counter = 0
grouped = pd.DataFrame()
for study in publications_dict.keys():
    for sector, sector_filter in filter_sector.items():
        for year_key, year_value in years_dict.items():
            
            filtered = parameters.loc[
                (parameters["study"] == study)
                & (parameters["Sektorenzuordnung"] == sector_filter)
                & (parameters["Jahr"].isin(year_value))
            ].dropna(axis=1, how="all")

            filtered = filtered.groupby("Prozesskategorie").agg({
                k: v for k, v in new_agg_dict.items() if k in filtered.columns
            })
            grouped = pd.concat([grouped, filtered])
            
            print(counter)
            counter += 1

Define functions for:
* Extracting data for a given parameter (group)
* Adding the value counts (no of studies / parameter values)
* Transposing to prepare for creating of boxplots

_Missing: groupby "Prozesskategorie" with can only be performed once every piece of information is in distinct columns / rows since otherwise it would be lost within the grouping process ..._

In [ ]:
def extract_info(df, cols_dict, parameter, sector, year):
    """Extract info for given parameter, sector and year from data"""
    filtered_df = df.loc[
        (df["Sektorenzuordnung"] == filter_sector[sector]) 
        & (df["Jahr"].isin(year))
    ]
    filtered_df = filtered_df[[
        col for col in filtered_df.columns 
        if col in cols_dict[parameter]]]
    
    return filtered_df

In [ ]:
def add_value_counts(filtered_df, cols_dict, parameter):
    """Add value counts to a filtered df"""
    filtered_df["number_publications"] = filtered_df.index.value_counts()
    
    sources_cols = [
        col for col in filtered_df.columns if "Fundstelle" in col
    ]
    filtered_df["number_entries"] = (
        filtered_df.count(axis=1) 
        - (len(sources_cols) 
           + len(["Jahr", "Sektorenzuordnung", "study"]))
    )
    entries = filtered_df["number_entries"].groupby(filtered_df.index).sum()
    filtered_df["abs_number_entries"] = entries
    
    for col in sources_cols:
        entries = filtered_df[col].groupby(filtered_df.index).apply(lambda x: ";".join(x))
        filtered_df["all_sources_" + col] = entries
    
    return filtered_df

In [ ]:
def transpose_and_dropna(filtered_df, cols_dict, parameter):
    """Transpose filtered df and prepare for stats extraction & plotting"""
    cols_to_drop = [
        col for col in filtered_df.columns if "Fundstelle" in col
    ]
    cols_to_drop.extend([
        "Jahr", "Sektorenzuordnung", "study",
        "number_publications", "number_entries", "abs_number_entries"
    ])
    transposed_df = filtered_df.T.drop([
        col for col in cols_to_drop if col in filtered_df.columns
    ])
    transposed_df.dropna(axis=1, how="all", inplace=True)
    
    return transposed_df

In [ ]:
testbert = extract_info(grouped, cols_dict, "variable Kosten", "ind", status_quo_years)

In [ ]:
testbert = add_value_counts(testbert, cols_dict, "variable Kosten")

In [ ]:
testbert.loc[testbert.index == "Primäraluminiumelektrolyse"]

In [ ]:
testbert_T = transpose_and_dropna(testbert, cols_dict, "variable_Kosten")

In [ ]:
testbert_T

In [ ]:
testbert_T.plot(kind="box")

# Eligibility for load shifting / load shedding

## Prepare data on eligibility
At first, it is analyzed whether a certain process is eligible for load shifting, load shedding or both according to the publications evaluated.
* Evaluate columns with information on eligibility per study and process
* Analyze the results per process over all studies (sum up absolute frequency)
* _NOTE_: There is no distinction made between sectors since it is assumed that studies equally treat a (cross-cutting) technology / process which is used across multiple sectors.

In [ ]:
eligibility_dict = {}
cols = cols_dict["Eignung"]

if not add_sources:
    cols = [el for el in cols if not "Fundstelle" in el]

for k, v in publications_dict.items():
    eligibility_dict[k] = v[[el for el in cols if el in v.columns]].drop_duplicates(subset="Prozess", keep="first")
    if add_sources:
        source_col = [el for el in eligibility_dict[k].columns if "Fundstelle" in el]
        eligibility_dict[k][source_col] = k.split("_")[0] + ", S. " + eligibility_dict[k][source_col].astype(str) + "; "

In [ ]:
# Show a sample eligibility DataFrame (example: Apel (2012))
eligibility_dict["Ape12"].head()

In [ ]:
shift_eligibility_df = pd.DataFrame(columns=["Prozess"])
shed_eligibility_df = pd.DataFrame(columns=["Prozess"])
inc_eligibility_df = pd.DataFrame(columns=["Prozess"])

# Iterate over all DataFrames and put them together
for k, v in eligibility_dict.items():
    shift_eligibility_df = pd.merge(shift_eligibility_df, v[["Prozess", "Lastverschiebung"]], on = "Prozess", suffixes=["", "_"+k], how="outer")
    shed_eligibility_df = pd.merge(shed_eligibility_df, v[["Prozess", "Lastverzicht"]], on = "Prozess", suffixes=["", "_"+k], how="outer")
    inc_eligibility_df = pd.merge(inc_eligibility_df, v[["Prozess", "Lasterhöhung"]], on = "Prozess", suffixes=["", "_"+k], how="outer")
    
if add_sources:
    source_cols = [el for el in shift_eligibility_df.columns if "Fundstelle" in el]
    shift_eligibility_df["sources"] = shift_eligibility_df[source_cols].fillna("").sum(axis=1)
    shift_eligibility_df = shift_eligibility_df.drop(source_cols, axis=1)

    source_cols = [el for el in shed_eligibility_df.columns if "Fundstelle" in el]
    shed_eligibility_df["sources"] = shed_eligibility_df[source_cols].fillna("").sum(axis=1)
    shed_eligibility_df = shed_eligibility_df.drop(source_cols, axis=1)
    
    source_cols = [el for el in inc_eligibility_df.columns if "Fundstelle" in el]
    inc_eligibility_df["sources"] = inc_eligibility_df[source_cols].fillna("").sum(axis=1)
    inc_eligibility_df = inc_eligibility_df.drop(source_cols, axis=1)

In [ ]:
shift_eligibility_df

Prepare data for plotting:
* Calculate the row sums, i.e. the number of publications that say, a certain process is eligible for load shifting / shedding / increase
* Calculate the number of zeros in a row, i.e. the number of publications that say, a certain process is not eligible for load shifting / shedding / increase
* Sort in descending order

In [ ]:
# Calculate the sum over all DataFrames and determine number of zeros
shift_eligibility_df = shift_eligibility_df.set_index("Prozess")
shed_eligibility_df = shed_eligibility_df.set_index("Prozess")
inc_eligibility_df = inc_eligibility_df.set_index("Prozess")

shift_eligibility_df["geeignet"] = shift_eligibility_df.sum(axis = "columns")
shift_eligibility_df["ungeeignet"] = shift_eligibility_df.count(axis = "columns", numeric_only=True) - 1 - shift_eligibility_df["geeignet"]
shed_eligibility_df["geeignet"] = shed_eligibility_df.sum(axis = "columns")
shed_eligibility_df["ungeeignet"] = shed_eligibility_df.count(axis = "columns", numeric_only=True) - 1 - shed_eligibility_df["geeignet"]
inc_eligibility_df["geeignet"] = inc_eligibility_df.sum(axis = "columns")
inc_eligibility_df["ungeeignet"] = inc_eligibility_df.count(axis = "columns", numeric_only=True) - 1 - inc_eligibility_df["geeignet"]

In [ ]:
shift_df = shift_eligibility_df[["geeignet", "ungeeignet"]]
shift_df["Summe"] = shift_df.sum(axis=1)
shift_df = shift_df.sort_values(by="Summe", ascending=False).drop("Summe", axis=1)

shed_df = shed_eligibility_df[["geeignet", "ungeeignet"]].reindex(shift_df.index)
inc_df = inc_eligibility_df[["geeignet", "ungeeignet"]].reindex(shift_df.index)

In [ ]:
shift_df

In [ ]:
%matplotlib inline

## Create plots for eligibility

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20,15), sharex = True)

_ = shift_df.plot(kind="bar", color=["dimgrey", "lightgrey"], stacked=True, ax=ax1)
_ = ax1.set_ylabel("absolute Häufigkeit")
_ = ax1.set_title("Lastverschiebung")
_ = ax1.set_yticks(np.arange(0, 19, 1))

_ = shed_df.plot(kind="bar", color=["dimgrey", "lightgrey"], stacked=True, ax=ax2)
_ = ax2.set_ylabel("absolute Häufigkeit")
_ = ax2.set_title("Lastverzicht")
_ = ax2.set_yticks(np.arange(0, 19, 1))

_ = inc_df.plot(kind="bar", color=["dimgrey", "lightgrey"], stacked=True, ax=ax3)
_ = ax3.set_ylabel("absolute Häufigkeit")
_ = ax3.set_title("Lasterhöhung")
_ = ax3.set_yticks(np.arange(0, 19, 1))

_ = plt.xlabel("Lastmanagementkategorie")
plt.savefig(path_folder_plots+"eligibility_all.png", dpi = 150, bbox_inches = "tight")

plt.show()

## Write the data

In [ ]:
eligibility_df = pd.concat([shift_df, shed_df, inc_df], keys=["Lastverschiebung", "Lastverzicht", "Lasterhöhung"], axis = "columns")

In [ ]:
eligibility_df.to_excel(path_folder_stats+"eligibility.xlsx")

if add_sources:
    eligibility_sources_dict = {"shift_eligibility": shift_eligibility_df["sources"].unique().sum(),
                                "shed_eligibility": shed_eligibility_df["sources"].unique().sum(),
                                "inc_eligibility": inc_eligibility_df["sources"].unique().sum()}
    
    if write_sources:
        save_sources(eligibility_sources_dict, path_folder_sources, "eligibility_sources.xlsx")

# Capacity-related parameters
In the following, all capacity-related demand response parameters are evaluated. These comprise
* demand response potentials in positive or negative direction (in MW),
* average, minimum and maximum load factors (in percent of installed capacity) as well as
* installed capacities (in MW) and shiftable load share (in percent).

The evaluation is carried out per process / appliance / branch (demand response category) for a given sector and year.

## Industry

### Positive load shift potential

Extract and process the information on the positive load shift potential:
* Obtain information from overall industry dictionary containing all parameters for the publications
* Transform data info into a dictionary of DataFrames with the processes as columns which is indexed by years
* Save the statistical information on the DataFrames into Excel workbooks
* Draw a boxplot (for the status quo)

In [ ]:
# Extract all columns containing potential information (mixing min / max / ave values)
pot_pos_cols = list(set(cols_dict["Potenzial positiv Mittel"]).union(
    set(cols_dict["Potenzial positiv min"])).union(
    set(cols_dict["Potenzial positiv max"])))

In [ ]:
# Extract info on positive demand reponse potentials and put the info from different publications together into one DataFrame
potential_pos_ind_ave_dict, potential_pos_ind_ave_df = extract_info(publications_ind_years_dict, cols_dict["Potenzial positiv Mittel"], 
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_pos_ind_min_dict, potential_pos_ind_min_df = extract_info(publications_ind_years_dict, cols_dict["Potenzial positiv min"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_pos_ind_max_dict, potential_pos_ind_max_df = extract_info(publications_ind_years_dict, cols_dict["Potenzial positiv max"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_pos_ind_overall_dict, potential_pos_ind_overall_df = extract_info(publications_ind_years_dict, pot_pos_cols, 
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")

In [ ]:
# Show original data shape (example: Apel et al. 2012, Status quo)
publications_ind_years_dict["Ape12_SQ"]

In [ ]:
# Show some excerpt of data from one publication (example: Steurer 2017, Status quo)
potential_pos_ind_ave_dict["Ste17_SQ"].head(8)

Create a DataFrames containing the potential information for the respective_years

In [ ]:
(potential_pos_ind_ave_years_dict, potential_pos_ind_min_years_dict, potential_pos_ind_max_years_dict, potential_pos_ind_overall_years_dict), \
(sources_potential_pos_ind_ave_dict, sources_potential_pos_ind_min_dict, sources_potential_pos_ind_max_dict, sources_potential_pos_ind_overall_dict) \
    = create_info_dict_for_years([potential_pos_ind_ave_df, potential_pos_ind_min_df, potential_pos_ind_max_df, potential_pos_ind_overall_df], years_dict, 
                                 negate=False, sample_stats=sample_stats, add_sources=True)

In [ ]:
# Save description of the DataFrames -> statistical information
if write_outputs:
    save_info_stats(potential_pos_ind_ave_years_dict, path_folder_stats, "potential_pos_ave_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_ind_min_years_dict, path_folder_stats, "potential_pos_min_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_ind_max_years_dict, path_folder_stats, "potential_pos_max_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_ind_overall_years_dict, path_folder_stats, "potential_pos_overall_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_potential_pos_ind_ave_dict, path_folder_sources, "potential_pos_ave_ind_sources.xlsx")
    save_sources(sources_potential_pos_ind_min_dict, path_folder_sources, "potential_pos_min_ind_sources.xlsx")
    save_sources(sources_potential_pos_ind_max_dict, path_folder_sources, "potential_pos_max_ind_sources.xlsx")
    save_sources(sources_potential_pos_ind_overall_dict, path_folder_sources, "potential_pos_overall_ind_sources.xlsx")

_NOTE: The following can be used to create nicer visualizations, e.g. for presentations:_
* _Swap order of appearance of columns (i.e. demand response categories)_
* _Plot only an excerpt of all categories_

In [ ]:
if presentation_stuff:
    # show original columns order
    cols_original = potential_pos_ind_ave_years_dict["SQ"].columns.to_list()
    for i, el in enumerate(cols_original):
        print(i, el)
    len(cols_original)

In [ ]:
if presentation_stuff:
    # create demanded columns order
    cols_demanded = cols_original[:7] + cols_original[8:9] + cols_original[11:12] + cols_original[7:8] + cols_original[9:11] + cols_original[12:13] + cols_original[13:] 
    len(cols_demanded)

In [ ]:
if presentation_stuff:
    # reassign the manipulated DataFrame
    potential_pos_ind_ave_years_dict["SQ"] = potential_pos_ind_ave_years_dict["SQ"][cols_demanded]

In [ ]:
if presentation_stuff:
    potential_pos_ind_ave_years_dict["SQ"] = potential_pos_ind_ave_years_dict["SQ"][cols_demanded[9:] + cols_demanded[:9]]

In [ ]:
if presentation_stuff:
    create_info_boxplot(potential_pos_ind_ave_years_dict, 
                        year="SQ",
                        title="Werte für das durchschnittliche positive Lastmanagementpotenzial (Lastreduktion)",
                        ylabel="durchschn. positives Potenzial in MW",
                        show_title=False,
                        colors=colors,
                        ylim=[0, 3000],
                        use_colors=use_colors,
                        use_limits=False,
                        sample_stats=sample_stats, 
                        add_sources=add_sources, 
                        swarmplot=swarmplot, 
                        savefig=True, 
                        filename="potential_pos_ave_ind_SQ_slice")

Plot the positive potential information as a box plot

In [ ]:
create_info_boxplot(potential_pos_ind_ave_years_dict, 
                    year="SQ",
                    title="Werte für das durchschnittliche positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="durchschn. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=True, 
                    filename="potential_pos_ave_ind_SQ")

In [ ]:
create_info_boxplot(potential_pos_ind_min_years_dict, 
                    year="SQ",
                    title="Werte für das minimale positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="min. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_min_ind_SQ")

In [ ]:
create_info_boxplot(potential_pos_ind_max_years_dict, 
                    year="SQ",
                    title="Werte für das maximale positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="max. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_max_ind_SQ")

In [ ]:
create_info_boxplot(potential_pos_ind_overall_years_dict, 
                    year="SQ",
                    title="Werte für das positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_overall_ind_SQ")

### Positive load shedding potential

Extract and process the information on the positive load shedding potential:
* Obtain information from overall industry dictionary containing all parameters for the publications
* Transform data info into a dictionary of DataFrames with the processes as columns which is indexed by years
* Save the statistical information on the DataFrames into Excel workbooks
* Draw a boxplot (for the status quo)

In [ ]:
# Extract all columns containing potential information (mixing min / max / ave values)
pot_pos_shed_cols = list(
    set(cols_dict["Potenzial positiv Mittel Lastverzicht"]).union(
        set(cols_dict["Potenzial positiv min Lastverzicht"])).union(
        set(cols_dict["Potenzial positiv max Lastverzicht"])))

In [ ]:
# Extract info on positive demand reponse potentials and put the info from different publications together into one DataFrame
potential_pos_ind_ave_shed_dict, potential_pos_ind_ave_shed_df = extract_info(
    publications_ind_years_dict, cols_dict["Potenzial positiv Mittel Lastverzicht"], 
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)
potential_pos_ind_min_shed_dict, potential_pos_ind_min_shed_df = extract_info(
    publications_ind_years_dict, cols_dict["Potenzial positiv min Lastverzicht"],
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)
potential_pos_ind_max_shed_dict, potential_pos_ind_max_shed_df = extract_info(
    publications_ind_years_dict, cols_dict["Potenzial positiv max Lastverzicht"],
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)
potential_pos_ind_overall_shed_dict, potential_pos_ind_overall_shed_df = extract_info(
    publications_ind_years_dict, pot_pos_shed_cols, 
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)

Create a DataFrames containing the potential information for the respective_years

In [ ]:
(potential_pos_ind_ave_shed_years_dict, 
 potential_pos_ind_min_shed_years_dict, 
 potential_pos_ind_max_shed_years_dict, 
 potential_pos_ind_overall_shed_years_dict), \
(sources_potential_pos_ind_ave_shed_dict, 
 sources_potential_pos_ind_min_shed_dict, 
 sources_potential_pos_ind_max_shed_dict, 
 sources_potential_pos_ind_overall_shed_dict) \
    = create_info_dict_for_years(
    [potential_pos_ind_ave_shed_df, potential_pos_ind_min_shed_df, 
     potential_pos_ind_max_shed_df, potential_pos_ind_overall_shed_df], years_dict, 
                                 negate=False, sample_stats=sample_stats, add_sources=True)

In [ ]:
# Save description of the DataFrames -> statistical information
if write_outputs:
    save_info_stats(potential_pos_ind_ave_shed_years_dict, path_folder_stats, "potential_pos_ave_shed_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_ind_min_shed_years_dict, path_folder_stats, "potential_pos_min_shed_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_ind_max_shed_years_dict, path_folder_stats, "potential_pos_max_shed_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_ind_overall_shed_years_dict, path_folder_stats, "potential_pos_overall_shed_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_potential_pos_ind_ave_shed_dict, path_folder_sources, "potential_pos_ave_shed_ind_sources.xlsx")
    save_sources(sources_potential_pos_ind_min_shed_dict, path_folder_sources, "potential_pos_min_shed_ind_sources.xlsx")
    save_sources(sources_potential_pos_ind_max_shed_dict, path_folder_sources, "potential_pos_max_shed_ind_sources.xlsx")
    save_sources(sources_potential_pos_ind_overall_shed_dict, path_folder_sources, "potential_pos_overall_shed_ind_sources.xlsx")

Plot the positive potential information as a box plot

In [ ]:
create_info_boxplot(potential_pos_ind_ave_shed_years_dict, 
                    year="SQ",
                    title="Werte für das durchschnittliche positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="durchschn. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=True, 
                    filename="potential_pos_ave_shed_ind_SQ")

In [ ]:
create_info_boxplot(potential_pos_ind_min_shed_years_dict, 
                    year="SQ",
                    title="Werte für das minimale positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="min. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_min_shed_ind_SQ")

In [ ]:
create_info_boxplot(potential_pos_ind_max_shed_years_dict, 
                    year="SQ",
                    title="Werte für das maximale positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="max. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_max_shed_ind_SQ")

In [ ]:
create_info_boxplot(potential_pos_ind_overall_shed_years_dict, 
                    year="SQ",
                    title="Werte für das positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_overall_shed_ind_SQ")

### Negative load shift potential

Extract the information on the negative load shift potential

In [ ]:
# Extract all columns containing potential information (mixing min / max / ave values)
pot_neg_cols = list(set(cols_dict["Potenzial negativ Mittel"]).union(set(cols_dict["Potenzial negativ min"])).union(set(cols_dict["Potenzial negativ max"])))

In [ ]:
potential_neg_ind_ave_dict, potential_neg_ind_ave_df = extract_info(publications_ind_years_dict, cols_dict["Potenzial negativ Mittel"], 
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_neg_ind_min_dict, potential_neg_ind_min_df = extract_info(publications_ind_years_dict, cols_dict["Potenzial negativ min"], 
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_neg_ind_max_dict, potential_neg_ind_max_df = extract_info(publications_ind_years_dict, cols_dict["Potenzial negativ max"], 
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_neg_ind_overall_dict, potential_neg_ind_overall_df = extract_info(publications_ind_years_dict, pot_neg_cols, 
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")

Create a DataFrames containing the potential information for the respective_years

In [ ]:
(potential_neg_ind_ave_years_dict, potential_neg_ind_min_years_dict, potential_neg_ind_max_years_dict, potential_neg_ind_overall_years_dict), \
(sources_potential_neg_ind_ave_dict, sources_potential_neg_ind_min_dict, sources_potential_neg_ind_max_dict, sources_potential_neg_ind_overall_dict) \
    = create_info_dict_for_years([potential_neg_ind_ave_df, potential_neg_ind_min_df, potential_neg_ind_max_df, potential_neg_ind_overall_df], years_dict, 
                                 negate=True, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save description of the DataFrames -> statistical information
if write_outputs:
    save_info_stats(potential_neg_ind_ave_years_dict, path_folder_stats, "potential_neg_ave_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_neg_ind_min_years_dict, path_folder_stats, "potential_neg_min_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_neg_ind_max_years_dict, path_folder_stats, "potential_neg_max_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_neg_ind_overall_years_dict, path_folder_stats, "potential_neg_overall_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_potential_neg_ind_ave_dict, path_folder_sources, "potential_neg_ave_ind_sources.xlsx")
    save_sources(sources_potential_neg_ind_min_dict, path_folder_sources, "potential_neg_min_ind_sources.xlsx")
    save_sources(sources_potential_neg_ind_max_dict, path_folder_sources, "potential_neg_max_ind_sources.xlsx")
    save_sources(sources_potential_neg_ind_overall_dict, path_folder_sources, "potential_neg_overall_ind_sources.xlsx")

In [ ]:
create_info_boxplot(potential_neg_ind_ave_years_dict, 
                    year="SQ",
                    title="Werte für das durchschnittliche negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="durchschn. negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-3000, 0],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_ave_ind_SQ")

In [ ]:
create_info_boxplot(potential_neg_ind_min_years_dict, 
                    year="SQ",
                    title="Werte für das minimale negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="min. negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-3000, 0],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_min_ind_SQ")

In [ ]:
create_info_boxplot(potential_neg_ind_max_years_dict, 
                    year="SQ",
                    title="Werte für das maximale negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="max. negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-3000, 0],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_max_ind_SQ")

In [ ]:
create_info_boxplot(potential_neg_ind_overall_years_dict, 
                    year="SQ",
                    title="Werte für das negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="negatives Potenzial in MW", 
                    show_title=False,
                    colors=colors,
                    ylim=[-3000, 0],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_overall_ind_SQ")

### Average load values

In [ ]:
ave_load_ind_dict, ave_load_ind_df = extract_info(publications_ind_years_dict, cols_dict["Durchschnittsauslastung"], 
                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
ave_load_ind_years_dict, sources_ave_load_ind_dict = create_info_dict_for_years(ave_load_ind_df, years_dict, 
                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(ave_load_ind_years_dict, path_folder_stats, "ave_load_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_ave_load_ind_dict, path_folder_sources, "ave_load_ind_sources.xlsx")

In [ ]:
create_info_boxplot(ave_load_ind_years_dict, 
                    year="SQ",
                    title="Durchschnittlicher Lastfaktor",
                    ylabel="durchschnittlicher Lastfaktor in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="ave_load_ind_SQ")

### Minimum load values

In [ ]:
min_load_ind_dict, min_load_ind_df = extract_info(publications_ind_years_dict, cols_dict["Minimalauslastung"],
                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
min_load_ind_years_dict, sources_min_load_ind_dict = create_info_dict_for_years(min_load_ind_df, years_dict, 
                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(min_load_ind_years_dict, path_folder_stats, "min_load_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_min_load_ind_dict, path_folder_sources, "min_load_ind_sources.xlsx")

In [ ]:
create_info_boxplot(min_load_ind_years_dict, 
                    year="SQ",
                    title="Minimaler Lastfaktor",
                    ylabel="minimaler Lastfaktor in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="min_load_ind_SQ")

### Maximum load values

In [ ]:
max_load_ind_dict, max_load_ind_df = extract_info(publications_ind_years_dict, cols_dict["Maximalauslastung"],
                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
max_load_ind_years_dict, sources_max_load_ind_dict = create_info_dict_for_years(max_load_ind_df, years_dict, 
                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(max_load_ind_years_dict, path_folder_stats, "max_load_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_max_load_ind_dict, path_folder_sources, "max_load_ind_sources.xlsx")

In [ ]:
create_info_boxplot(max_load_ind_years_dict, 
                    year="SQ",
                    title="Maximaler Lastfaktor",
                    ylabel="maximaler Lastfaktor in p.u.", 
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="max_load_ind_SQ")

### Installed capacity

In [ ]:
installed_cap_ind_dict, installed_cap_ind_df = extract_info(publications_ind_years_dict, cols_dict["installierte Leistung"],
                                                            drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                            drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")

In [ ]:
installed_cap_ind_years_dict, sources_installed_cap_ind_dict = create_info_dict_for_years(installed_cap_ind_df, years_dict, 
                                                          negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(installed_cap_ind_years_dict, path_folder_stats, "installed_cap_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_installed_cap_ind_dict, path_folder_sources, "installed_cap_ind_sources.xlsx")

In [ ]:
create_info_boxplot(installed_cap_ind_years_dict, 
                    year="SQ",
                    title="installierte Leistung",
                    ylabel="installierte Leistung in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="installed_cap_ind_SQ")

### Shiftable share

In [ ]:
shiftable_share_ind_dict, shiftable_share_ind_df = extract_info(publications_ind_years_dict, cols_dict["flexibilisierbarer Leistungsanteil"],
                                                                drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
shiftable_share_ind_dict["Ape12_SQ"]

In [ ]:
shiftable_share_ind_years_dict, sources_shiftable_share_ind_dict = create_info_dict_for_years(shiftable_share_ind_df, years_dict, 
                                                            negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(shiftable_share_ind_years_dict, path_folder_stats, "shiftable_share_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_shiftable_share_ind_dict, path_folder_sources, "shiftable_share_ind_sources.xlsx")

In [ ]:
create_info_boxplot(shiftable_share_ind_years_dict, 
                    year="SQ",
                    title="flexibilisierbarer Leistungsanteil",
                    ylabel="flexibilisierbarer Leistungsanteil in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1.5],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="shiftable_share_ind_SQ")

## Trade, commerce and services

### Positive load shift potential

Extract the information on the positive load shift potential

In [ ]:
# Extract info on positive demand reponse potentials and put the info from different publications together into one DataFrame
potential_pos_tcs_ave_dict, potential_pos_tcs_ave_df = extract_info(publications_tcs_years_dict, cols_dict["Potenzial positiv Mittel"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_pos_tcs_min_dict, potential_pos_tcs_min_df = extract_info(publications_tcs_years_dict, cols_dict["Potenzial positiv min"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_pos_tcs_max_dict, potential_pos_tcs_max_df = extract_info(publications_tcs_years_dict, cols_dict["Potenzial positiv max"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_pos_tcs_overall_dict, potential_pos_tcs_overall_df = extract_info(publications_tcs_years_dict, pot_pos_cols, 
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")

Create a DataFrames containing the potential information for the respective_years

In [ ]:
(potential_pos_tcs_ave_years_dict, potential_pos_tcs_min_years_dict, potential_pos_tcs_max_years_dict, potential_pos_tcs_overall_years_dict), \
(sources_potential_pos_tcs_ave_dict, sources_potential_pos_tcs_min_dict, sources_potential_pos_tcs_max_dict, sources_potential_pos_tcs_overall_dict) \
    = create_info_dict_for_years([potential_pos_tcs_ave_df, potential_pos_tcs_min_df, potential_pos_tcs_max_df, potential_pos_tcs_overall_df], years_dict, 
                                 negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Describe the DataFrame -> statistical information
if write_outputs:
    save_info_stats(potential_pos_tcs_ave_years_dict, path_folder_stats, "potential_pos_ave_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_tcs_min_years_dict, path_folder_stats, "potential_pos_min_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_tcs_max_years_dict, path_folder_stats, "potential_pos_max_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_tcs_overall_years_dict, path_folder_stats, "potential_pos_overall_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_potential_pos_tcs_ave_dict, path_folder_sources, "potential_pos_ave_tcs_sources.xlsx")
    save_sources(sources_potential_pos_tcs_min_dict, path_folder_sources, "potential_pos_min_tcs_sources.xlsx")
    save_sources(sources_potential_pos_tcs_max_dict, path_folder_sources, "potential_pos_max_tcs_sources.xlsx")
    save_sources(sources_potential_pos_tcs_overall_dict, path_folder_sources, "potential_pos_overall_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(potential_pos_tcs_ave_years_dict, 
                    year="SQ",
                    title="Werte für das durchschnittliche positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="durchschn. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_ave_tcs_SQ")

In [ ]:
create_info_boxplot(potential_pos_tcs_min_years_dict, 
                    year="SQ",
                    title="Werte für das minimale positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="min. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_min_tcs_SQ")

In [ ]:
create_info_boxplot(potential_pos_tcs_max_years_dict, 
                    year="SQ",
                    title="Werte für das maximale positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="max. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_max_tcs_SQ")

In [ ]:
create_info_boxplot(potential_pos_tcs_overall_years_dict, 
                    year="SQ",
                    title="Werte für das positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_overall_tcs_SQ")

### Positive load shedding potential

Extract the information on the positive load shedding potential

In [ ]:
# Extract info on positive demand reponse potentials and put the info from different publications together into one DataFrame
potential_pos_tcs_ave_shed_dict, potential_pos_tcs_ave_shed_df = extract_info(
    publications_tcs_years_dict, cols_dict["Potenzial positiv Mittel Lastverzicht"],
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)
potential_pos_tcs_min_shed_dict, potential_pos_tcs_min_shed_df = extract_info(
    publications_tcs_years_dict, cols_dict["Potenzial positiv min Lastverzicht"],
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)
potential_pos_tcs_max_shed_dict, potential_pos_tcs_max_shed_df = extract_info(
    publications_tcs_years_dict, cols_dict["Potenzial positiv max Lastverzicht"],
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)
potential_pos_tcs_overall_shed_dict, potential_pos_tcs_overall_shed_df = extract_info(
    publications_tcs_years_dict, pot_pos_shed_cols, 
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)

Create a DataFrames containing the potential information for the respective_years

In [ ]:
(potential_pos_tcs_ave_shed_years_dict, 
 potential_pos_tcs_min_shed_years_dict, 
 potential_pos_tcs_max_shed_years_dict, 
 potential_pos_tcs_overall_shed_years_dict), \
(sources_potential_pos_tcs_ave_shed_dict, 
 sources_potential_pos_tcs_min_shed_dict, 
 sources_potential_pos_tcs_max_shed_dict, 
 sources_potential_pos_tcs_overall_shed_dict) \
    = create_info_dict_for_years(
    [potential_pos_tcs_ave_shed_df, 
     potential_pos_tcs_min_shed_df, 
     potential_pos_tcs_max_shed_df, 
     potential_pos_tcs_overall_shed_df], years_dict, 
                                 negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Describe the DataFrame -> statistical information
if write_outputs:
    save_info_stats(potential_pos_tcs_ave_shed_years_dict, path_folder_stats, "potential_pos_ave_shed_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_tcs_min_shed_years_dict, path_folder_stats, "potential_pos_min_shed_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_tcs_max_shed_years_dict, path_folder_stats, "potential_pos_max_shed_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_tcs_overall_shed_years_dict, path_folder_stats, "potential_pos_overall_shed_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_potential_pos_tcs_ave_shed_dict, path_folder_sources, "potential_pos_ave_shed_tcs_sources.xlsx")
    save_sources(sources_potential_pos_tcs_min_shed_dict, path_folder_sources, "potential_pos_min_shed_tcs_sources.xlsx")
    save_sources(sources_potential_pos_tcs_max_shed_dict, path_folder_sources, "potential_pos_max_shed_tcs_sources.xlsx")
    save_sources(sources_potential_pos_tcs_overall_shed_dict, path_folder_sources, "potential_pos_overall_shed_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(potential_pos_tcs_ave_shed_years_dict, 
                    year="SQ",
                    title="Werte für das durchschnittliche positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="durchschn. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_ave_shed_tcs_SQ")

In [ ]:
create_info_boxplot(potential_pos_tcs_min_shed_years_dict, 
                    year="SQ",
                    title="Werte für das minimale positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="min. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_min_shed_tcs_SQ")

In [ ]:
create_info_boxplot(potential_pos_tcs_max_shed_years_dict, 
                    year="SQ",
                    title="Werte für das maximale positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="max. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_max_shed_tcs_SQ")

In [ ]:
create_info_boxplot(potential_pos_tcs_overall_shed_years_dict, 
                    year="SQ",
                    title="Werte für das positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_overall_shed_tcs_SQ")

### Negative load shift potential

Extract the information on the negative load shift potential

In [ ]:
# Extract info on negative demand reponse potentials and put the info from different publications together into one DataFrame
potential_neg_tcs_ave_dict, potential_neg_tcs_ave_df = extract_info(publications_tcs_years_dict, cols_dict["Potenzial negativ Mittel"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_neg_tcs_min_dict, potential_neg_tcs_min_df = extract_info(publications_tcs_years_dict, cols_dict["Potenzial negativ min"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_neg_tcs_max_dict, potential_neg_tcs_max_df = extract_info(publications_tcs_years_dict, cols_dict["Potenzial negativ max"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_neg_tcs_overall_dict, potential_neg_tcs_overall_df = extract_info(publications_tcs_years_dict, pot_neg_cols, 
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")

Create a DataFrames containing the potential information for the respective_years

In [ ]:
(potential_neg_tcs_ave_years_dict, potential_neg_tcs_min_years_dict, potential_neg_tcs_max_years_dict, potential_neg_tcs_overall_years_dict), \
(sources_potential_neg_tcs_ave_dict, sources_potential_neg_tcs_min_dict, sources_potential_neg_tcs_max_dict, sources_potential_neg_tcs_overall_dict) \
    = create_info_dict_for_years([potential_neg_tcs_ave_df, potential_neg_tcs_min_df, potential_neg_tcs_max_df, potential_neg_tcs_overall_df], years_dict, 
                                 negate=True, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Describe the DataFrame -> statistical information
if write_outputs:
    save_info_stats(potential_neg_tcs_ave_years_dict, path_folder_stats, "potential_neg_ave_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_neg_tcs_min_years_dict, path_folder_stats, "potential_neg_min_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_neg_tcs_max_years_dict, path_folder_stats, "potential_neg_max_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_neg_tcs_overall_years_dict, path_folder_stats, "potential_neg_overall_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_potential_neg_tcs_ave_dict, path_folder_sources, "potential_neg_ave_tcs_sources.xlsx")
    save_sources(sources_potential_neg_tcs_min_dict, path_folder_sources, "potential_neg_min_tcs_sources.xlsx")
    save_sources(sources_potential_neg_tcs_max_dict, path_folder_sources, "potential_neg_max_tcs_sources.xlsx")
    save_sources(sources_potential_neg_tcs_overall_dict, path_folder_sources, "potential_neg_overall_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(potential_neg_tcs_ave_years_dict, 
                    year="SQ",
                    title="Werte für das durchschnittliche negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="durchschn. negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-3000, 0],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_ave_tcs_SQ")

In [ ]:
create_info_boxplot(potential_neg_tcs_min_years_dict, 
                    year="SQ",
                    title="Werte für das minimale negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="min. negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-3000, 0],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_min_tcs_SQ")

In [ ]:
create_info_boxplot(potential_neg_tcs_max_years_dict, 
                    year="SQ",
                    title="Werte für das maximale negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="max. negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-10000, 0],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_max_tcs_SQ")

In [ ]:
create_info_boxplot(potential_neg_tcs_overall_years_dict, 
                    year="SQ",
                    title="Werte für das negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-10000, 0],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_overall_tcs_SQ")

### Average load values

In [ ]:
ave_load_tcs_dict, ave_load_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Durchschnittsauslastung"],
                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
ave_load_tcs_years_dict, sources_ave_load_tcs_dict = create_info_dict_for_years(ave_load_tcs_df, years_dict, 
                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(ave_load_tcs_years_dict, path_folder_stats, "ave_load_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_ave_load_tcs_dict, path_folder_sources, "ave_load_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(ave_load_tcs_years_dict, 
                    year="SQ",
                    title="Durchschnittlicher Lastfaktor",
                    ylabel="durchschnittlicher Lastfaktor in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="ave_load_tcs_SQ")

### Minimum load values

In [ ]:
min_load_tcs_dict, min_load_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Minimalauslastung"],
                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
min_load_tcs_years_dict, sources_min_load_tcs_dict = create_info_dict_for_years(min_load_tcs_df, years_dict, 
                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(min_load_tcs_years_dict, path_folder_stats, "min_load_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_min_load_tcs_dict, path_folder_sources, "min_load_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(min_load_tcs_years_dict, 
                    year="SQ",
                    title="Minimaler Lastfaktor",
                    ylabel="minimaler Lastfaktor in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="min_load_tcs_SQ")

### Maximum load values

In [ ]:
max_load_tcs_dict, max_load_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Maximalauslastung"],
                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
max_load_tcs_years_dict, sources_max_load_tcs_dict = create_info_dict_for_years(max_load_tcs_df, years_dict, 
                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(max_load_tcs_years_dict, path_folder_stats, "max_load_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_max_load_tcs_dict, path_folder_sources, "max_load_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(max_load_tcs_years_dict, 
                    year="SQ",
                    title="Maximaler Lastfaktor",
                    ylabel="maximaler Lastfaktor in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="max_load_tcs_SQ")

### Installed capacity

In [ ]:
installed_cap_tcs_dict, installed_cap_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["installierte Leistung"],
                                                            drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                            drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")

In [ ]:
installed_cap_tcs_years_dict, sources_installed_cap_tcs_dict = create_info_dict_for_years(installed_cap_tcs_df, years_dict, 
                                                          negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(installed_cap_tcs_years_dict, path_folder_stats, "installed_cap_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_installed_cap_tcs_dict, path_folder_sources, "installed_cap_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(installed_cap_tcs_years_dict, 
                    year="SQ",
                    title="installierte Leistung",
                    ylabel="installierte Leistung in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="installed_cap_tcs_SQ")

### Shiftable share

In [ ]:
shiftable_share_tcs_dict, shiftable_share_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["flexibilisierbarer Leistungsanteil"],
                                                                drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
shiftable_share_tcs_years_dict, sources_shiftable_share_tcs_dict = create_info_dict_for_years(shiftable_share_tcs_df, years_dict, 
                                                            negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(shiftable_share_tcs_years_dict, path_folder_stats, "shiftable_share_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_shiftable_share_tcs_dict, path_folder_sources, "shiftable_share_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(shiftable_share_tcs_years_dict, 
                    year="SQ",
                    title="flexibilisierbarer Leistungsanteil",
                    ylabel="flexibilisierbarer Leistungsanteil in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1.5],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="shiftable_share_tcs_SQ")

## Households

### Positive load shift potential

Extract the information on the positive load shift potential

In [ ]:
# Extract info on positive demand reponse potentials and put the info from different publications together into one DataFrame
potential_pos_hoho_ave_dict, potential_pos_hoho_ave_df = extract_info(publications_hoho_years_dict, cols_dict["Potenzial positiv Mittel"],
                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_pos_hoho_min_dict, potential_pos_hoho_min_df = extract_info(publications_hoho_years_dict, cols_dict["Potenzial positiv min"],
                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_pos_hoho_max_dict, potential_pos_hoho_max_df = extract_info(publications_hoho_years_dict, cols_dict["Potenzial positiv max"],
                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_pos_hoho_overall_dict, potential_pos_hoho_overall_df = extract_info(publications_hoho_years_dict, pot_pos_cols, 
                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")

Create a DataFrames containing the potential information for the respective_years

In [ ]:
(potential_pos_hoho_ave_years_dict, potential_pos_hoho_min_years_dict, potential_pos_hoho_max_years_dict, potential_pos_hoho_overall_years_dict), \
(sources_potential_pos_hoho_ave_dict, sources_potential_pos_hoho_min_dict, sources_potential_pos_hoho_max_dict, sources_potential_pos_hoho_overall_dict) \
    = create_info_dict_for_years([potential_pos_hoho_ave_df, potential_pos_hoho_min_df, potential_pos_hoho_max_df, potential_pos_hoho_overall_df], years_dict, 
                                 negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Describe the DataFrame -> statistical information
if write_outputs:
    save_info_stats(potential_pos_hoho_ave_years_dict, path_folder_stats, "potential_pos_ave_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_hoho_min_years_dict, path_folder_stats, "potential_pos_min_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_hoho_max_years_dict, path_folder_stats, "potential_pos_max_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_hoho_overall_years_dict, path_folder_stats, "potential_pos_overall_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_potential_pos_hoho_ave_dict, path_folder_sources, "potential_pos_ave_hoho_sources.xlsx")
    save_sources(sources_potential_pos_hoho_min_dict, path_folder_sources, "potential_pos_min_hoho_sources.xlsx")
    save_sources(sources_potential_pos_hoho_max_dict, path_folder_sources, "potential_pos_max_hoho_sources.xlsx")
    save_sources(sources_potential_pos_hoho_overall_dict, path_folder_sources, "potential_pos_overall_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(potential_pos_hoho_ave_years_dict, 
                    year="SQ",
                    title="Werte für das durchschnittliche positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="durchschn. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_ave_hoho_SQ")

In [ ]:
create_info_boxplot(potential_pos_hoho_min_years_dict, 
                    year="SQ",
                    title="Werte für das minimale positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="min. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_min_hoho_SQ")

In [ ]:
create_info_boxplot(potential_pos_hoho_max_years_dict, 
                    year="SQ",
                    title="Werte für das maximale positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="max. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_max_hoho_SQ")

In [ ]:
create_info_boxplot(potential_pos_hoho_overall_years_dict, 
                    year="SQ",
                    title="Werte für das positive Lastmanagementpotenzial (Lastreduktion)",
                    ylabel="positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_overall_hoho_SQ")

### Positive load shedding potential

Extract the information on the positive load shedding potential

In [ ]:
# Extract info on positive demand reponse potentials and put the info from different publications together into one DataFrame
potential_pos_hoho_ave_shed_dict, potential_pos_hoho_ave_shed_df = extract_info(
    publications_hoho_years_dict, cols_dict["Potenzial positiv Mittel Lastverzicht"],
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)
potential_pos_hoho_min_shed_dict, potential_pos_hoho_min_shed_df = extract_info(
    publications_hoho_years_dict, cols_dict["Potenzial positiv min Lastverzicht"],
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)
potential_pos_hoho_max_shed_dict, potential_pos_hoho_max_shed_df = extract_info(
    publications_hoho_years_dict, cols_dict["Potenzial positiv max Lastverzicht"],
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)
potential_pos_hoho_overall_shed_dict, potential_pos_hoho_overall_shed_df = extract_info(
    publications_hoho_years_dict, pot_pos_shed_cols, 
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum"
)

Create a DataFrames containing the potential information for the respective_years

In [ ]:
(potential_pos_hoho_ave_shed_years_dict, 
 potential_pos_hoho_min_shed_years_dict, 
 potential_pos_hoho_max_shed_years_dict, 
 potential_pos_hoho_overall_shed_years_dict), \
(sources_potential_pos_hoho_ave_shed_dict, 
 sources_potential_pos_hoho_min_shed_dict, 
 sources_potential_pos_hoho_max_shed_dict, 
 sources_potential_pos_hoho_overall_shed_dict) \
    = create_info_dict_for_years([
    potential_pos_hoho_ave_shed_df, 
    potential_pos_hoho_min_shed_df, 
    potential_pos_hoho_max_shed_df, 
    potential_pos_hoho_overall_shed_df], years_dict, 
                                 negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Describe the DataFrame -> statistical information
if write_outputs:
    save_info_stats(potential_pos_hoho_ave_shed_years_dict, path_folder_stats, "potential_pos_ave_shed_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_hoho_min_shed_years_dict, path_folder_stats, "potential_pos_min_shed_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_hoho_max_shed_years_dict, path_folder_stats, "potential_pos_max_shed_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_pos_hoho_overall_shed_years_dict, path_folder_stats, "potential_pos_overall_shed_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_potential_pos_hoho_ave_shed_dict, path_folder_sources, "potential_pos_ave_shed_hoho_sources.xlsx")
    save_sources(sources_potential_pos_hoho_min_shed_dict, path_folder_sources, "potential_pos_min_shed_hoho_sources.xlsx")
    save_sources(sources_potential_pos_hoho_max_shed_dict, path_folder_sources, "potential_pos_max_shed_hoho_sources.xlsx")
    save_sources(sources_potential_pos_hoho_overall_shed_dict, path_folder_sources, "potential_pos_overall_shed_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(potential_pos_hoho_ave_shed_years_dict, 
                    year="SQ",
                    title="Werte für das durchschnittliche positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="durchschn. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_ave_shed_hoho_SQ")

In [ ]:
create_info_boxplot(potential_pos_hoho_min_shed_years_dict, 
                    year="SQ",
                    title="Werte für das minimale positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="min. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_min_shed_hoho_SQ")

In [ ]:
create_info_boxplot(potential_pos_hoho_max_shed_years_dict, 
                    year="SQ",
                    title="Werte für das maximale positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="max. positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_max_shed_hoho_SQ")

In [ ]:
create_info_boxplot(potential_pos_hoho_overall_shed_years_dict, 
                    year="SQ",
                    title="Werte für das positive Abschaltpotenzial (Lastverzicht)",
                    ylabel="positives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_pos_overall_shed_hoho_SQ")

### Negative load shift potential

Extract the information on the negative load shift potential

In [ ]:
# Extract info on negative demand reponse potentials and put the info from different publications together into one DataFrame
potential_neg_hoho_ave_dict, potential_neg_hoho_ave_df = extract_info(publications_hoho_years_dict, cols_dict["Potenzial negativ Mittel"],
                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_neg_hoho_min_dict, potential_neg_hoho_min_df = extract_info(publications_hoho_years_dict, cols_dict["Potenzial negativ min"],
                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_neg_hoho_max_dict, potential_neg_hoho_max_df = extract_info(publications_hoho_years_dict, cols_dict["Potenzial negativ max"],
                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")
potential_neg_hoho_overall_dict, potential_neg_hoho_overall_df = extract_info(publications_hoho_years_dict, pot_neg_cols, 
                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")

Create a DataFrames containing the potential information for the respective_years

In [ ]:
(potential_neg_hoho_ave_years_dict, potential_neg_hoho_min_years_dict, potential_neg_hoho_max_years_dict, potential_neg_hoho_overall_years_dict), \
(sources_potential_neg_hoho_ave_dict, sources_potential_neg_hoho_min_dict, sources_potential_neg_hoho_max_dict, sources_potential_neg_hoho_overall_dict) \
    = create_info_dict_for_years([potential_neg_hoho_ave_df, potential_neg_hoho_min_df, potential_neg_hoho_max_df, potential_neg_hoho_overall_df], years_dict, 
                                 negate=True, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save description of the DataFrames -> statistical information
if write_outputs:
    save_info_stats(potential_neg_hoho_ave_years_dict, path_folder_stats, "potential_neg_ave_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_neg_hoho_min_years_dict, path_folder_stats, "potential_neg_min_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_neg_hoho_max_years_dict, path_folder_stats, "potential_neg_max_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)
    save_info_stats(potential_neg_hoho_overall_years_dict, path_folder_stats, "potential_neg_overall_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_potential_neg_hoho_ave_dict, path_folder_sources, "potential_neg_ave_hoho_sources.xlsx")
    save_sources(sources_potential_neg_hoho_min_dict, path_folder_sources, "potential_neg_min_hoho_sources.xlsx")
    save_sources(sources_potential_neg_hoho_max_dict, path_folder_sources, "potential_neg_max_hoho_sources.xlsx")
    save_sources(sources_potential_neg_hoho_overall_dict, path_folder_sources, "potential_neg_overall_hoho_sources.xlsx")

_NOTE: The following can be used to create nicer visualizations, e.g. for presentations:_
* _Swap order of appearance of columns (i.e. demand response categories)_
* _Plot only an excerpt of all categories_

In [ ]:
if presentation_stuff:
    # show original columns order
    cols_original = potential_neg_hoho_ave_years_dict["SQ"].columns.to_list()
    for i, el in enumerate(cols_original):
        print(i, el)
    len(cols_original)

In [ ]:
if presentation_stuff:
    # create demanded columns order
    cols_demanded = cols_original[4:10] + cols_original[:4] + cols_original[10:]
    len(cols_demanded)

In [ ]:
if presentation_stuff:
    # reassign the manipulated DataFrame
    potential_neg_hoho_ave_years_dict["SQ"] = potential_neg_hoho_ave_years_dict["SQ"][cols_demanded]

In [ ]:
create_info_boxplot(potential_neg_hoho_ave_years_dict, 
                    year="SQ",
                    title="Werte für das durchschnittliche negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="durchschn. negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-10000, 0],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_ave_hoho_SQ")

In [ ]:
create_info_boxplot(potential_neg_hoho_min_years_dict, 
                    year="SQ",
                    title="Werte für das minimale negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="min. negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-3000, 0],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_min_hoho_SQ")

In [ ]:
create_info_boxplot(potential_neg_hoho_max_years_dict, 
                    year="SQ",
                    title="Werte für das maximale negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="max. negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-3000, 0],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_max_hoho_SQ")

In [ ]:
create_info_boxplot(potential_neg_hoho_overall_years_dict, 
                    year="SQ",
                    title="Werte für das negative Lastmanagementpotenzial (Lasterhöhung)",
                    ylabel="negatives Potenzial in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[-30000, 0],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="potential_neg_overall_hoho_SQ")

### Average load values

In [ ]:
ave_load_hoho_dict, ave_load_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Durchschnittsauslastung"],
                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
ave_load_hoho_years_dict, sources_ave_load_hoho_dict = create_info_dict_for_years(ave_load_hoho_df, years_dict, 
                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(ave_load_hoho_years_dict, path_folder_stats, "ave_load_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_ave_load_hoho_dict, path_folder_sources, "ave_load_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(ave_load_hoho_years_dict, 
                    year="SQ",
                    title="Durchschnittlicher Lastfaktor",
                    ylabel="durchschnittlicher Lastfaktor in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="ave_load_hoho_SQ")

### Minimum load values

In [ ]:
min_load_hoho_dict, min_load_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Minimalauslastung"],
                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
min_load_hoho_years_dict, sources_min_load_hoho_dict = create_info_dict_for_years(min_load_hoho_df, years_dict, 
                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(min_load_hoho_years_dict, path_folder_stats, "min_load_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_min_load_hoho_dict, path_folder_sources, "min_load_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(min_load_hoho_years_dict, 
                    year="SQ",
                    title="Minimaler Lastfaktor",
                    ylabel="minimaler Lastfaktor in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="min_load_hoho_SQ")

### Maximum load values

In [ ]:
max_load_hoho_dict, max_load_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Maximalauslastung"],
                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
max_load_hoho_years_dict, sources_max_load_hoho_dict = create_info_dict_for_years(max_load_hoho_df, years_dict, 
                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(max_load_hoho_years_dict, path_folder_stats, "max_load_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_max_load_hoho_dict, path_folder_sources, "max_load_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(max_load_hoho_years_dict, 
                    year="SQ",
                    title="Maximaler Lastfaktor",
                    ylabel="maximaler Lastfaktor in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="max_load_hoho_SQ")

### Installed capacity

In [ ]:
installed_cap_hoho_dict, installed_cap_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["installierte Leistung"],
                                                              drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                              drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="sum")

In [ ]:
installed_cap_hoho_years_dict, sources_installed_cap_hoho_dict = create_info_dict_for_years(installed_cap_hoho_df, years_dict, 
                                                           negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(installed_cap_hoho_years_dict, path_folder_stats, "installed_cap_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_installed_cap_hoho_dict, path_folder_sources, "installed_cap_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(installed_cap_hoho_years_dict, 
                    year="SQ",
                    title="installierte Leistung",
                    ylabel="installierte Leistung in MW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="installed_cap_hoho_SQ")

### Shiftable share

In [ ]:
shiftable_share_hoho_dict, shiftable_share_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["flexibilisierbarer Leistungsanteil"],
                                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
shiftable_share_hoho_years_dict, sources_shiftable_share_hoho_dict = create_info_dict_for_years(shiftable_share_hoho_df, years_dict, 
                                                             negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(shiftable_share_hoho_years_dict, path_folder_stats, "shiftable_share_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_shiftable_share_hoho_dict, path_folder_sources, "shiftable_share_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(shiftable_share_hoho_years_dict, 
                    year="SQ",
                    title="flexibilisierbarer Leistungsanteil",
                    ylabel="flexibilisierbarer Leistungsanteil in p.u.",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 1.5],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="shiftable_share_hoho_SQ")

# Time-related parameters
In the following, all time-related demand response parameters are evaluated. These comprise
* activation duration (in h),
* interference duration in postive and negative direction (in h),
* shifting duration (in h),
* regeneration duration (in h) as well as
* maximum activations per year resp. week (in 1).

The evaluation is carried out per process / appliance for a given sector and year.

## Industry

### Activation duration

In [ ]:
activation_duration_ind_dict, activation_duration_ind_df = extract_info(publications_ind_years_dict, cols_dict["Aktivierungsdauer"],
                                                                        drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                        drop_datalack=drop_datalack, modify_categories=modify_categories, agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
activation_duration_ind_years_dict, sources_activation_duration_ind_dict = create_info_dict_for_years(activation_duration_ind_df, years_dict, 
                                                                negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(activation_duration_ind_years_dict, path_folder_stats, "activation_duration_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_activation_duration_ind_dict, path_folder_sources, "activation_duration_ind_sources.xlsx")

In [ ]:
create_info_boxplot(activation_duration_ind_years_dict, 
                    year="SQ",
                    title="Aktivierungsdauer",
                    ylabel="Aktivierungsdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="activation_duration_ind_SQ")

### interference duration positive load shifting

In [ ]:
interference_duration_pos_ind_dict, interference_duration_pos_ind_df = extract_info(publications_ind_years_dict, cols_dict["Schaltdauer positiv"],
                                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                    agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
interference_duration_pos_ind_years_dict, sources_interference_duration_pos_ind_dict = create_info_dict_for_years(interference_duration_pos_ind_df, years_dict, 
                                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(interference_duration_pos_ind_years_dict, path_folder_stats, "interference_duration_pos_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_interference_duration_pos_ind_dict, path_folder_sources, "interference_duration_pos_ind_sources.xlsx")

In [ ]:
create_info_boxplot(interference_duration_pos_ind_years_dict, 
                    year="SQ",
                    title="positive Schaltdauer (Lastreduktion)",
                    ylabel="positive Schaltdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="interference_duration_pos_ind_SQ")

### interference duration positive load shedding

In [ ]:
interference_duration_pos_shed_ind_dict, interference_duration_pos_shed_ind_df = extract_info(
    publications_ind_years_dict, cols_dict["Schaltdauer positiv Lastverzicht"],
    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
    drop_datalack=drop_datalack, modify_categories=modify_categories, 
    agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
interference_duration_pos_shed_ind_years_dict, sources_interference_duration_pos_shed_ind_dict = create_info_dict_for_years(interference_duration_pos_shed_ind_df, years_dict, 
                                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(interference_duration_pos_shed_ind_years_dict, path_folder_stats, "interference_duration_pos_shed_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_interference_duration_pos_shed_ind_dict, path_folder_sources, "interference_duration_pos_shed_ind_sources.xlsx")

In [ ]:
create_info_boxplot(interference_duration_pos_shed_ind_years_dict, 
                    year="SQ",
                    title="positive Schaltdauer (Lastverzicht)",
                    ylabel="positive Schaltdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="interference_duration_pos_shed_ind_SQ")

### interference duration negative

In [ ]:
interference_duration_neg_ind_dict, interference_duration_neg_ind_df = extract_info(publications_ind_years_dict, cols_dict["Schaltdauer negativ"],
                                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                    agg_rule="weighted_average", weights=pot_neg_cols)

In [ ]:
interference_duration_neg_ind_years_dict, sources_interference_duration_neg_ind_dict = create_info_dict_for_years(interference_duration_neg_ind_df, years_dict, 
                                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(interference_duration_neg_ind_years_dict, path_folder_stats, "interference_duration_neg_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_interference_duration_neg_ind_dict, path_folder_sources, "interference_duration_neg_ind_sources.xlsx")

In [ ]:
create_info_boxplot(interference_duration_neg_ind_years_dict, 
                    year="SQ",
                    title="negative Schaltdauer (Lasterhöhung)",
                    ylabel="negative Schaltdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="interference_duration_neg_ind_SQ")

### shifting duration

In [ ]:
shifting_duration_ind_dict, shifting_duration_ind_df = extract_info(publications_ind_years_dict, cols_dict["Verschiebedauer"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                    agg_rule="weighted_average", weights=pot_neg_cols)

In [ ]:
shifting_duration_ind_years_dict, sources_shifting_duration_ind_dict = create_info_dict_for_years(shifting_duration_ind_df, years_dict, 
                                                              negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(shifting_duration_ind_years_dict, path_folder_stats, "shifting_duration_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_shifting_duration_ind_dict, path_folder_sources, "shifting_duration_ind_sources.xlsx")

In [ ]:
create_info_boxplot(shifting_duration_ind_years_dict, 
                    year="SQ",
                    title="Verschiebedauer",
                    ylabel="Verschiebedauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 75],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="shifting_duration_ind_SQ")

### regeneration duration

In [ ]:
regeneration_duration_ind_dict, regeneration_duration_ind_df = extract_info(publications_ind_years_dict, cols_dict["Regenerationsdauer"],
                                                                            drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                            drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                            agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
regeneration_duration_ind_years_dict, sources_regeneration_duration_ind_dict = create_info_dict_for_years(regeneration_duration_ind_df, years_dict, 
                                                                  negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(regeneration_duration_ind_years_dict, path_folder_stats, "regeneration_duration_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_regeneration_duration_ind_dict, path_folder_sources, "regeneration_duration_ind_sources.xlsx")

In [ ]:
create_info_boxplot(regeneration_duration_ind_years_dict, 
                    year="SQ",
                    title="Regenerierungsdauer (Dauer zwischen zwei Aktivierungen)",
                    ylabel="Regenerierungsdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="regeneration_duration_ind_SQ")

### maximum activations per year load shifting

In [ ]:
maximum_activations_year_ind_dict, maximum_activations_year_ind_df = extract_info(publications_ind_years_dict, cols_dict["maximale Abrufhäufigkeit pro Jahr"],
                                                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                  agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
maximum_activations_year_ind_years_dict, sources_maximum_activations_year_ind_dict = create_info_dict_for_years(maximum_activations_year_ind_df, years_dict, 
                                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(maximum_activations_year_ind_years_dict, path_folder_stats, "maximum_activations_year_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_maximum_activations_year_ind_dict, path_folder_sources, "maximum_activations_year_ind_sources.xlsx")

In [ ]:
create_info_boxplot(maximum_activations_year_ind_years_dict, 
                    year="SQ",
                    title="Maximale Aktivierungshäufigkeit pro Jahr",
                    ylabel="Anzahl an Aktivierungen in 1/a",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="maximum_activations_year_ind_SQ")

### maximum activations per year load shedding

In [ ]:
maximum_activations_year_shed_ind_dict, maximum_activations_year_shed_ind_df = extract_info(publications_ind_years_dict, cols_dict["maximale Abrufhäufigkeit pro Jahr Lastverzicht"],
                                                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                  agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
maximum_activations_year_shed_ind_years_dict, sources_maximum_activations_year_shed_ind_dict = create_info_dict_for_years(maximum_activations_year_shed_ind_df, years_dict, 
                                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(maximum_activations_year_shed_ind_years_dict, path_folder_stats, "maximum_activations_year_shed_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_maximum_activations_year_shed_ind_dict, path_folder_sources, "maximum_activations_year_shed_ind_sources.xlsx")

In [ ]:
create_info_boxplot(maximum_activations_year_shed_ind_years_dict, 
                    year="SQ",
                    title="Maximale Aktivierungshäufigkeit pro Jahr (Lastverzicht)",
                    ylabel="Anzahl an Aktivierungen in 1/a",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="maximum_activations_year_shed_ind_SQ")

## Trade, commerce and services

### Activation duration

In [ ]:
activation_duration_tcs_dict, activation_duration_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Aktivierungsdauer"],
                                                                        drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                        drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                        agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
activation_duration_tcs_years_dict, sources_activation_duration_tcs_dict = create_info_dict_for_years(activation_duration_tcs_df, years_dict, 
                                                                negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(activation_duration_tcs_years_dict, path_folder_stats, "activation_duration_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_activation_duration_tcs_dict, path_folder_sources, "activation_duration_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(activation_duration_tcs_years_dict, 
                    year="SQ",
                    title="Aktivierungsdauer",
                    ylabel="Aktivierungsdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="activation_duration_tcs_SQ")

### interference duration positive load shifting

In [ ]:
interference_duration_pos_tcs_dict, interference_duration_pos_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Schaltdauer positiv"],
                                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                    agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
interference_duration_pos_tcs_years_dict, sources_interference_duration_pos_tcs_dict = create_info_dict_for_years(interference_duration_pos_tcs_df, years_dict, 
                                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(interference_duration_pos_tcs_years_dict, path_folder_stats, "interference_duration_pos_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_interference_duration_pos_tcs_dict, path_folder_sources, "interference_duration_pos_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(interference_duration_pos_tcs_years_dict, 
                    year="SQ",
                    title="positive Schaltdauer (Lastreduktion)",
                    ylabel="positive Schaltdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="interference_duration_pos_tcs_SQ")

### interference duration positive load shedding

In [ ]:
interference_duration_pos_shed_tcs_dict, interference_duration_pos_shed_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Schaltdauer positiv Lastverzicht"],
                                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                    agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
interference_duration_pos_shed_tcs_years_dict, sources_interference_duration_pos_shed_tcs_dict = create_info_dict_for_years(interference_duration_pos_shed_tcs_df, years_dict, 
                                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(interference_duration_pos_shed_tcs_years_dict, path_folder_stats, "interference_duration_pos_shed_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_interference_duration_pos_shed_tcs_dict, path_folder_sources, "interference_duration_pos_shed_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(interference_duration_pos_shed_tcs_years_dict, 
                    year="SQ",
                    title="positive Schaltdauer (Lastverzicht)",
                    ylabel="positive Schaltdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="interference_duration_pos_shed_tcs_SQ")

### interference duration negative

In [ ]:
interference_duration_neg_tcs_dict, interference_duration_neg_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Schaltdauer negativ"],
                                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                    agg_rule="weighted_average", weights=pot_neg_cols)

In [ ]:
interference_duration_neg_tcs_years_dict, sources_interference_duration_neg_tcs_dict = create_info_dict_for_years(interference_duration_neg_tcs_df, years_dict, 
                                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(interference_duration_neg_tcs_years_dict, path_folder_stats, "interference_duration_neg_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_interference_duration_neg_tcs_dict, path_folder_sources, "interference_duration_neg_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(interference_duration_neg_tcs_years_dict, 
                    year="SQ",
                    title="negative Schaltdauer (Lasterhöhung)",
                    ylabel="negative Schaltdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="interference_duration_neg_tcs_SQ")

### shifting duration

In [ ]:
shifting_duration_tcs_dict, shifting_duration_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Verschiebedauer"],
                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                    agg_rule="weighted_average", weights=pot_neg_cols)

In [ ]:
shifting_duration_tcs_years_dict, sources_shifting_duration_tcs_dict = create_info_dict_for_years(shifting_duration_tcs_df, years_dict, 
                                                              negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(shifting_duration_tcs_years_dict, path_folder_stats, "shifting_duration_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_shifting_duration_tcs_dict, path_folder_sources, "shifting_duration_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(shifting_duration_tcs_years_dict, 
                    year="SQ",
                    title="Verschiebedauer",
                    ylabel="Verschiebedauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 75],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="shifting_duration_tcs_SQ")

### regeneration duration

In [ ]:
regeneration_duration_tcs_dict, regeneration_duration_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Regenerationsdauer"],
                                                                            drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                            drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                            agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
regeneration_duration_tcs_years_dict, sources_regeneration_duration_tcs_dict = create_info_dict_for_years(regeneration_duration_tcs_df, years_dict, 
                                                                  negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(regeneration_duration_tcs_years_dict, path_folder_stats, "regeneration_duration_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_regeneration_duration_tcs_dict, path_folder_sources, "regeneration_duration_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(regeneration_duration_tcs_years_dict, 
                    year="SQ",
                    title="Regenerierungsdauer (Dauer zwischen zwei Aktivierungen)",
                    ylabel="Regenerierungsdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="regeneration_duration_tcs_SQ")

### maximum activations per year load shifting

In [ ]:
maximum_activations_year_tcs_dict, maximum_activations_year_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["maximale Abrufhäufigkeit pro Jahr"],
                                                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                  agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
maximum_activations_year_tcs_years_dict, sources_maximum_activations_year_tcs_dict = create_info_dict_for_years(maximum_activations_year_tcs_df, years_dict, 
                                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(maximum_activations_year_tcs_years_dict, path_folder_stats, "maximum_activations_year_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_maximum_activations_year_tcs_dict, path_folder_sources, "maximum_activations_year_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(maximum_activations_year_tcs_years_dict, 
                    year="SQ",
                    title="Maximale Aktivierungshäufigkeit pro Jahr",
                    ylabel="Anzahl an Aktivierungen in 1/a",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="maximum_activations_year_tcs_SQ")

### maximum activations per year load shedding

In [ ]:
maximum_activations_year_shed_tcs_dict, maximum_activations_year_shed_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["maximale Abrufhäufigkeit pro Jahr Lastverzicht"],
                                                                                  drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                  drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                  agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
maximum_activations_year_shed_tcs_years_dict, sources_maximum_activations_year_shed_tcs_dict = create_info_dict_for_years(maximum_activations_year_shed_tcs_df, years_dict, 
                                                                     negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(maximum_activations_year_shed_tcs_years_dict, path_folder_stats, "maximum_activations_year_shed_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_maximum_activations_shed_year_tcs_dict, path_folder_sources, "maximum_activations_year_shed_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(maximum_activations_year_shed_tcs_years_dict, 
                    year="SQ",
                    title="Maximale Aktivierungshäufigkeit pro Jahr (Lastverzicht)",
                    ylabel="Anzahl an Aktivierungen in 1/a",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="maximum_activations_year_shed_tcs_SQ")

## Households

### Activation duration

In [ ]:
activation_duration_hoho_dict, activation_duration_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Aktivierungsdauer"],
                                                                          drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                          drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                          agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
activation_duration_hoho_years_dict, sources_activation_duration_hoho_dict = create_info_dict_for_years(activation_duration_hoho_df, years_dict, 
                                                                 negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(activation_duration_hoho_years_dict, path_folder_stats, "activation_duration_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_activation_duration_hoho_dict, path_folder_sources, "activation_duration_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(activation_duration_hoho_years_dict, 
                    year="SQ",
                    title="Aktivierungsdauer",
                    ylabel="Aktivierungsdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="activation_duration_hoho_SQ")

### interference duration positive load shifting

In [ ]:
interference_duration_pos_hoho_dict, interference_duration_pos_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Schaltdauer positiv"],
                                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                      agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
interference_duration_pos_hoho_years_dict, sources_interference_duration_pos_hoho_dict = create_info_dict_for_years(interference_duration_pos_hoho_df, years_dict, 
                                                                       negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(interference_duration_pos_hoho_years_dict, path_folder_stats, "interference_duration_pos_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_interference_duration_pos_hoho_dict, path_folder_sources, "interference_duration_pos_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(interference_duration_pos_hoho_years_dict, 
                    year="SQ",
                    title="positive Schaltdauer (Lastreduktion)",
                    ylabel="positive Schaltdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="interference_duration_pos_hoho_SQ")

### interference duration positive load shedding

In [ ]:
interference_duration_pos_shed_hoho_dict, interference_duration_pos_shed_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Schaltdauer positiv Lastverzicht"],
                                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                      agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
interference_duration_pos_shed_hoho_years_dict, sources_interference_duration_pos_shed_hoho_dict = create_info_dict_for_years(interference_duration_pos_shed_hoho_df, years_dict, 
                                                                       negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(interference_duration_pos_shed_hoho_years_dict, path_folder_stats, "interference_duration_pos_shed_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_interference_duration_pos_shed_hoho_dict, path_folder_sources, "interference_duration_pos_shed_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(interference_duration_pos_shed_hoho_years_dict, 
                    year="SQ",
                    title="positive Schaltdauer (Lastverzicht)",
                    ylabel="positive Schaltdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="interference_duration_pos_shed_hoho_SQ")

### interference duration negative

In [ ]:
interference_duration_neg_hoho_dict, interference_duration_neg_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Schaltdauer negativ"],
                                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                      agg_rule="weighted_average", weights=pot_neg_cols)

In [ ]:
interference_duration_neg_hoho_years_dict, sources_interference_duration_neg_hoho_dict = create_info_dict_for_years(interference_duration_neg_hoho_df, years_dict, 
                                                                       negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(interference_duration_neg_hoho_years_dict, path_folder_stats, "interference_duration_neg_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_interference_duration_neg_hoho_dict, path_folder_sources, "interference_duration_neg_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(interference_duration_neg_hoho_years_dict, 
                    year="SQ",
                    title="negative Schaltdauer (Lasterhöhung)",
                    ylabel="negative Schaltdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="interference_duration_neg_hoho_SQ")

### shifting duration

In [ ]:
shifting_duration_hoho_dict, shifting_duration_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Verschiebedauer"],
                                                                      drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                      drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                      agg_rule="weighted_average", weights=pot_neg_cols)

In [ ]:
shifting_duration_hoho_years_dict, sources_shifting_duration_hoho_dict = create_info_dict_for_years(shifting_duration_hoho_df, years_dict, 
                                                               negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(shifting_duration_hoho_years_dict, path_folder_stats, "shifting_duration_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_shifting_duration_hoho_dict, path_folder_sources, "shifting_duration_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(shifting_duration_hoho_years_dict, 
                    year="SQ",
                    title="Verschiebedauer",
                    ylabel="Verschiebedauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 75],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="shifting_duration_hoho_SQ")

### regeneration duration

In [ ]:
regeneration_duration_hoho_dict, regeneration_duration_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Regenerationsdauer"],
                                                                              drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                              drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                              agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
regeneration_duration_hoho_years_dict, sources_regeneration_duration_hoho_dict = create_info_dict_for_years(regeneration_duration_hoho_df, years_dict, 
                                                                   negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(regeneration_duration_hoho_years_dict, path_folder_stats, "regeneration_duration_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_regeneration_duration_hoho_dict, path_folder_sources, "regeneration_duration_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(regeneration_duration_hoho_years_dict, 
                    year="SQ",
                    title="Regenerierungsdauer (Dauer zwischen zwei Aktivierungen)",
                    ylabel="Regenerierungsdauer in h",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="regeneration_duration_hoho_SQ")

### maximum activations per year load shifting

In [ ]:
maximum_activations_year_hoho_dict, maximum_activations_year_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["maximale Abrufhäufigkeit pro Jahr"],
                                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                    agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
maximum_activations_year_hoho_years_dict, sources_maximum_activations_year_hoho_dict = create_info_dict_for_years(maximum_activations_year_hoho_df, years_dict, 
                                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(maximum_activations_year_hoho_years_dict, path_folder_stats, "maximum_activations_year_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_maximum_activations_year_hoho_dict, path_folder_sources, "maximum_activations_year_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(maximum_activations_year_hoho_years_dict, 
                    year="SQ",
                    title="Maximale Aktivierungshäufigkeit pro Jahr",
                    ylabel="Anzahl an Aktivierungen in 1/a",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="maximum_activations_year_hoho_SQ")

### maximum activations per year load shedding

In [ ]:
maximum_activations_year_shed_hoho_dict, maximum_activations_year_shed_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["maximale Abrufhäufigkeit pro Jahr Lastverzicht"],
                                                                                    drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                                    drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                                    agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
maximum_activations_year_shed_hoho_years_dict, sources_maximum_activations_year_shed_hoho_dict = create_info_dict_for_years(maximum_activations_year_shed_hoho_df, years_dict, 
                                                                      negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(maximum_activations_year_shed_hoho_years_dict, path_folder_stats, "maximum_activations_year_shed_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_maximum_activations_year_shed_hoho_dict, path_folder_sources, "maximum_activations_year_shed_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(maximum_activations_year_shed_hoho_years_dict, 
                    year="SQ",
                    title="Maximale Aktivierungshäufigkeit pro Jahr (Lastverzicht)",
                    ylabel="Anzahl an Aktivierungen in 1/a",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 10000],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="maximum_activations_year_shed_hoho_SQ")

# Cost-related parameters
In the following, all cost-related demand response parameters are evaluated. These comprise
* specific investments (in $€_{2018}/kW$),
* variable costs (in $€_{2018}/MWh$) as well as
* fixed costs (in $€_{2018}/kW$).

The evaluation is carried out per process / appliance for a given sector and year.

## Industry

### Specific investments

In [ ]:
specific_investments_ind_dict, specific_investments_ind_df = extract_info(publications_ind_years_dict, cols_dict["Investitionsausgaben"],
                                                                          drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                          drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                          agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
specific_investments_ind_years_dict, sources_specific_investments_ind_dict = create_info_dict_for_years(specific_investments_ind_df, years_dict, 
                                                                 negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(specific_investments_ind_years_dict, path_folder_stats, "specific_investments_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_specific_investments_ind_dict, path_folder_sources, "specific_investments_ind_sources.xlsx")

In [ ]:
create_info_boxplot(specific_investments_ind_years_dict, 
                    year="SQ",
                    title="Spezifische Investitionsausgaben",
                    ylabel="spezifische Investitionsausgaben in €_2018/kW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 2000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="specific_investments_ind_SQ")

### variable costs load shifting

In [ ]:
variable_costs_ind_dict, variable_costs_ind_df = extract_info(publications_ind_years_dict, cols_dict["variable Kosten"],
                                                              drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                              drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                              agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
variable_costs_ind_years_dict, sources_variable_costs_ind_dict = create_info_dict_for_years(variable_costs_ind_df, years_dict, 
                                                           negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(variable_costs_ind_years_dict, path_folder_stats, "variable_costs_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_variable_costs_ind_dict, path_folder_sources, "variable_costs_ind_sources.xlsx")

In [ ]:
create_info_boxplot(variable_costs_ind_years_dict, 
                    year="SQ",
                    title="Variable Kosten",
                    ylabel="variable Kosten in €_2018/MWh",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="variable_costs_ind_SQ")

In [ ]:
variable_costs_ind_years_dict

### variable costs load shedding

In [ ]:
variable_costs_ind_dict, variable_costs_ind_df = extract_info(publications_ind_years_dict, cols_dict["variable Kosten Lastverzicht"],
                                                              drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                              drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                              agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
variable_costs_ind_years_dict, sources_variable_costs_ind_dict = create_info_dict_for_years(variable_costs_ind_df, years_dict, 
                                                           negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(variable_costs_ind_years_dict, path_folder_stats, "variable_costs_shedding_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_variable_costs_ind_dict, path_folder_sources, "variable_costs_ind_shedding_sources.xlsx")

In [ ]:
create_info_boxplot(variable_costs_ind_years_dict, 
                    year="SQ",
                    title="Variable Kosten Lastverzicht",
                    ylabel="variable Kosten in €_2018/MWh",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="variable_costs_shedding_ind_SQ")

### fixed costs

In [ ]:
fixed_costs_ind_dict, fixed_costs_ind_df = extract_info(publications_ind_years_dict, cols_dict["fixe Kosten"],
                                                        drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                        drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                        agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
fixed_costs_ind_years_dict, sources_fixed_costs_ind_dict = create_info_dict_for_years(fixed_costs_ind_df, years_dict, 
                                                        negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(fixed_costs_ind_years_dict, path_folder_stats, "fixed_costs_ind_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_fixed_costs_ind_dict, path_folder_sources, "fixed_costs_ind_sources.xlsx")

In [ ]:
create_info_boxplot(fixed_costs_ind_years_dict, 
                    year="SQ",
                    title="Fixe Kosten",
                    ylabel="fixe Kosten in €_2018/kW*a",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="fixed_costs_ind_SQ")

## Trade, commerce and services

### Specific investments

In [ ]:
specific_investments_tcs_dict, specific_investments_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["Investitionsausgaben"],
                                                                          drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                          drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                          agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
specific_investments_tcs_years_dict, sources_specific_investments_tcs_dict = create_info_dict_for_years(specific_investments_tcs_df, years_dict, 
                                                                 negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(specific_investments_tcs_years_dict, path_folder_stats, "specific_investments_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_specific_investments_tcs_dict, path_folder_sources, "specific_investments_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(specific_investments_tcs_years_dict, 
                    year="SQ",
                    title="Spezifische Investitionsausgaben",
                    ylabel="spezifische Investitionsausgaben in €_2018/kW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 2000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="specific_investments_tcs_SQ")

### variable costs load shifting

In [ ]:
variable_costs_tcs_dict, variable_costs_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["variable Kosten"],
                                                              drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                              drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                              agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
variable_costs_tcs_years_dict, sources_variable_costs_tcs_dict = create_info_dict_for_years(variable_costs_tcs_df, years_dict, 
                                                           negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(variable_costs_tcs_years_dict, path_folder_stats, "variable_costs_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_variable_costs_tcs_dict, path_folder_sources, "variable_costs_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(variable_costs_tcs_years_dict, 
                    year="SQ",
                    title="Variable Kosten",
                    ylabel="variable Kosten in €_2018/MWh",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="variable_costs_tcs_SQ")

### variable costs load shedding

In [ ]:
variable_costs_tcs_dict, variable_costs_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["variable Kosten Lastverzicht"],
                                                              drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                              drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                              agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
variable_costs_tcs_years_dict, sources_variable_costs_tcs_dict = create_info_dict_for_years(variable_costs_tcs_df, years_dict, 
                                                           negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(variable_costs_tcs_years_dict, path_folder_stats, "variable_costs_shedding_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_variable_costs_tcs_dict, path_folder_sources, "variable_costs_shedding_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(variable_costs_tcs_years_dict, 
                    year="SQ",
                    title="Variable Kosten Lastverzicht",
                    ylabel="variable Kosten in €_2018/MWh",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="variable_costs_shedding_tcs_SQ")

### fixed costs

In [ ]:
fixed_costs_tcs_dict, fixed_costs_tcs_df = extract_info(publications_tcs_years_dict, cols_dict["fixe Kosten"],
                                                        drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                        drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                        agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
fixed_costs_tcs_years_dict, sources_fixed_costs_tcs_dict = create_info_dict_for_years(fixed_costs_tcs_df, years_dict, 
                                                        negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(fixed_costs_tcs_years_dict, path_folder_stats, "fixed_costs_tcs_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_fixed_costs_tcs_dict, path_folder_sources, "fixed_costs_tcs_sources.xlsx")

In [ ]:
create_info_boxplot(fixed_costs_tcs_years_dict, 
                    year="SQ",
                    title="Fixe Kosten",
                    ylabel="fixe Kosten in €_2018/kW*a",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="fixed_costs_tcs_SQ")

## Households

### Specific investments

In [ ]:
specific_investments_hoho_dict, specific_investments_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["Investitionsausgaben"],
                                                                            drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                            drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                            agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
specific_investments_hoho_years_dict, sources_specific_investments_hoho_dict = create_info_dict_for_years(specific_investments_hoho_df, years_dict, 
                                                                  negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(specific_investments_hoho_years_dict, path_folder_stats, "specific_investments_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_specific_investments_hoho_dict, path_folder_sources, "specific_investments_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(specific_investments_hoho_years_dict, 
                    year="SQ",
                    title="Spezifische Investitionsausgaben",
                    ylabel="spez. Investitionsausgaben in €_2018/kW",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 2000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="specific_investments_hoho_SQ")

### variable costs load shifting

In [ ]:
variable_costs_hoho_dict, variable_costs_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["variable Kosten"],
                                                                drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
variable_costs_hoho_years_dict, sources_variable_costs_hoho_dict = create_info_dict_for_years(variable_costs_hoho_df, years_dict, 
                                                            negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(variable_costs_hoho_years_dict, path_folder_stats, "variable_costs_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_variable_costs_hoho_dict, path_folder_sources, "variable_costs_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(variable_costs_hoho_years_dict, 
                    year="SQ",
                    title="Variable Kosten",
                    ylabel="variable Kosten in €_2018/MWh",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="variable_costs_hoho_SQ")

### variable costs load shedding

In [ ]:
variable_costs_hoho_dict, variable_costs_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["variable Kosten Lastverzicht"],
                                                                drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                                drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                                agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
variable_costs_hoho_years_dict, sources_variable_costs_hoho_dict = create_info_dict_for_years(variable_costs_hoho_df, years_dict, 
                                                            negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(variable_costs_hoho_years_dict, path_folder_stats, "variable_costs_shedding_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_variable_costs_hoho_dict, path_folder_sources, "variable_costs_shedding_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(variable_costs_hoho_years_dict, 
                    year="SQ",
                    title="Variable Kosten Lastverzicht",
                    ylabel="variable Kosten in €_2018/MWh",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 3000],
                    use_colors=use_colors,
                    use_limits=False,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="variable_costs_shedding_hoho_SQ")

### fixed costs

In [ ]:
fixed_costs_hoho_dict, fixed_costs_hoho_df = extract_info(publications_hoho_years_dict, cols_dict["fixe Kosten"],
                                                          drop=drop, sample_stats=sample_stats, add_sources=add_sources,
                                                          drop_datalack=drop_datalack, modify_categories=modify_categories, 
                                                          agg_rule="weighted_average", weights=pot_pos_cols)

In [ ]:
fixed_costs_hoho_years_dict, sources_fixed_costs_hoho_dict = create_info_dict_for_years(fixed_costs_hoho_df, years_dict, 
                                                         negate=False, sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
if write_outputs:
    save_info_stats(fixed_costs_hoho_years_dict, path_folder_stats, "fixed_costs_hoho_stats.xlsx", sample_stats=sample_stats, add_sources=add_sources)

In [ ]:
# Save sources
if write_sources:
    save_sources(sources_fixed_costs_hoho_dict, path_folder_sources, "fixed_costs_hoho_sources.xlsx")

In [ ]:
create_info_boxplot(fixed_costs_hoho_years_dict, 
                    year="SQ",
                    title="Fixe Kosten",
                    ylabel="fixe Kosten in €_2018/kW*a",
                    show_title=False,
                    colors=colors,
                    ylim=[0, 24],
                    use_colors=use_colors,
                    use_limits=use_limits,
                    sample_stats=sample_stats, 
                    add_sources=add_sources, 
                    swarmplot=swarmplot, 
                    savefig=savefig, 
                    filename="fixed_costs_hoho_SQ")

# Potential outlook
In this section, an outlook for the future development of some Demand Response parameters up to (at most) 2050 is given.
Future development for the following parameters is assessed
* Capcity-related parameters: positive load shift potential, negative load shift potential
* Time-related parameters: ?
* Cost-related parameters: specific investments, variable costs and fixed costs

## Industry

### Select some processes for further analysis
There are two ways to determine, which processes resp. appliances shall be further evaluated:
* one rather systematic one choosing those with the highest potential (in the status quo resp. in 2050) and
* a self-defined selection of processes which is influenced by the literature itself (and a potential personal bias).
It can be chosen between the two by using the control parameter `use_nlargest_cols`.

Method and criterions used for selection when using the columns with the highest potentials:
* Select those with highest demand response potential in the status quo.
* Select those with highest demand response potential in the 2050.
* Combine the two, i.e. remove those that only show large potentials in one of the categories.
* Analysis is separately carried out for positive and negative potentials to select the top ones for either category.
* For the time- and cost-related parameters, the top ones of every category (pos / neg) is chosen.
* The number of processes resp. appliances to be considered is flexibly modified in order to receive 3 to 4 remaining processes resp. appliances.

_Note: A suitable alternative approach would be to choose those categories for which data coverage is best or for which differences in the future potential can be noticed. This is not yet implemented._

In [ ]:
# Get the 5 processes with the largest positive potential for the status quo and 2050
high_potential_pos_ind_cols_SQ = get_nlargest(potential_pos_ind_ave_years_dict, year="SQ", metric=metric, n=9, sample_stats=sample_stats)
high_potential_pos_ind_cols_2050 = get_nlargest(potential_pos_ind_ave_years_dict, year="2050", metric=metric, n=9, sample_stats=sample_stats)

# Get the 5 processes with the largest negative potential for the status quo and 2050
high_potential_neg_ind_cols_SQ = get_nlargest(potential_neg_ind_ave_years_dict, year="SQ", metric=metric, n=n, sample_stats=sample_stats)
high_potential_neg_ind_cols_2050 = get_nlargest(potential_neg_ind_ave_years_dict, year="2050", metric=metric, n=n, sample_stats=sample_stats)

# Select the processes with the largest potential by combining the two
high_potential_pos_ind_cols = list(set(high_potential_pos_ind_cols_SQ).intersection(set(high_potential_pos_ind_cols_2050)))
high_potential_neg_ind_cols = list(set(high_potential_neg_ind_cols_SQ).intersection(set(high_potential_neg_ind_cols_2050)))

print(high_potential_pos_ind_cols)
print(high_potential_neg_ind_cols)

In [ ]:
# Get the 3 processes with the largest positive potential for the status quo and 2050
highest_potential_pos_ind_cols_SQ = get_nlargest(potential_pos_ind_ave_years_dict, year="SQ", metric=metric, n=3, sample_stats=sample_stats)
highest_potential_pos_ind_cols_2050 = get_nlargest(potential_pos_ind_ave_years_dict, year="2050", metric=metric, n=3, sample_stats=sample_stats)

# Get the 3 processes with the largest negative potential for the status quo and 2050
highest_potential_neg_ind_cols_SQ = get_nlargest(potential_neg_ind_ave_years_dict, year="SQ", metric=metric, n=3, sample_stats=sample_stats)
highest_potential_neg_ind_cols_2050 = get_nlargest(potential_neg_ind_ave_years_dict, year="2050", metric=metric, n=3, sample_stats=sample_stats)

# Select the processes with the largest potential by combining the two
highest_potential_ind_cols = list(set(highest_potential_pos_ind_cols_SQ).intersection(set(highest_potential_pos_ind_cols_2050)))
highest_potential_neg_ind_cols = list(set(highest_potential_neg_ind_cols_SQ).intersection(set(highest_potential_neg_ind_cols_2050)))
highest_potential_ind_cols.extend(highest_potential_neg_ind_cols)
highest_potential_ind_cols = list(set(highest_potential_ind_cols))

print(highest_potential_ind_cols)

Alternative approach:
* No systematic choice of processes, but self-selection of those of interest.
* Variable values are overwritten such that the function calls themselves do not have to be adjusted and flexibility is increased.

In [ ]:
if not use_nlargest_cols:
    if not modify_categories:
        industry_cols = ["Chlor-Alkali-Elektrolyse", "Elektrostahlherstellung", "Primäraluminiumelektrolyse", "Holzstoffherstellung"]
    elif aggregation_mode == "granular":
        industry_cols = ["Chlor-Alkali-Elektrolyse", "Elektrostahlherstellung (Lichtbogenofen)", "Primäraluminiumelektrolyse", "Holz- und Zellstoffherstellung"]
    elif aggregation_mode == "broad":
        industry_cols = ["Chlor-Alkali-Elektrolyse", "Elektrostahlherstellung (Lichtbogenofen)", "Primäraluminiumelektrolyse", "Papierherstellung"]
    high_potential_pos_ind_cols = industry_cols
    high_potential_neg_ind_cols = industry_cols
    highest_potential_ind_cols = industry_cols
    print(industry_cols)

### Positive load shift potential

Create a dict containing the statistical info per year as well as a DataFrame containing the info on the development of the respective parameter

In [ ]:
potential_pos_ind_ave_projection_dict, potential_pos_ind_ave_projection_df = extract_stats(potential_pos_ind_ave_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(potential_pos_ind_ave_df, years_dict, high_potential_pos_ind_cols, 
                          title="Positives Lastmanagementpotenzial (Lastreduktion) für ausgewählte Industrieprozesse - Projektion",
                          ylabel="Potenzial in MW", 
                          colors=colors,
                          ylim=[0, 3000],
                          use_colors=use_colors,
                          use_limits=False, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="potential_pos_ind_ave")

In [ ]:
if write_sources:
    save_future_sources(potential_pos_ind_ave_df, high_potential_pos_ind_cols, path_folder=path_folder_sources, filename="potential_pos_ind_ave_projection_sources.xlsx",  
                        add_sources=add_sources)

### Negative load shift potential

In [ ]:
potential_neg_ind_ave_projection_dict, potential_neg_ind_ave_projection_df = extract_stats(potential_neg_ind_ave_years_dict, years_dict)

In [ ]:
create_projection_boxplot(potential_neg_ind_ave_df, years_dict, high_potential_neg_ind_cols, 
                          title="Negatives Lastmanagementpotenzial (Lasterhöhung) für ausgewählte Industrieprozesse - Projektion",
                          ylabel="Potenzial in MW", 
                          colors=colors,
                          ylim=[-3000, 0],
                          use_colors=use_colors,
                          negate=True,
                          use_limits=False, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="potential_neg_ind_ave")

In [ ]:
if write_sources:
    save_future_sources(potential_neg_ind_ave_df, high_potential_neg_ind_cols, path_folder=path_folder_sources, filename="potential_neg_ind_ave_projection_sources.xlsx", 
                        add_sources=add_sources)

Visualize the data using a grouped boxplot

### Specific investments

In [ ]:
specific_investments_ind_projection_dict, specific_investments_ind_projection_df = extract_stats(specific_investments_ind_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(specific_investments_ind_df, years_dict, highest_potential_ind_cols, 
                          title="Spezifische Investitionsausgaben für ausgewählte Industrieprozesse - Projektion",
                          ylabel="spezifische Investitionsausgaben in €_2018/kW", 
                          colors=colors,
                          ylim=[0, 3000],
                          use_colors=use_colors,
                          use_limits=use_limits, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="specific_investments_ind")

In [ ]:
if write_sources:
    save_future_sources(specific_investments_ind_df, highest_potential_ind_cols, path_folder=path_folder_sources, filename="specific_investments_ind_projection_sources.xlsx",  
                        add_sources=add_sources)

### Variable costs

In [ ]:
variable_costs_ind_projection_dict, variable_costs_ind_projection_df = extract_stats(variable_costs_ind_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(variable_costs_ind_df, years_dict, highest_potential_ind_cols, 
                          title="Variable Kosten für ausgewählte Industrieprozesse - Projektion", 
                          ylabel="variable Kosten in €_2018/MWh", 
                          colors=colors,
                          ylim=[0, 3000],
                          use_colors=use_colors,
                          use_limits=use_limits, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="variable_costs_ind")

In [ ]:
if write_sources:
    save_future_sources(variable_costs_ind_df, highest_potential_ind_cols, path_folder=path_folder_sources, filename="variable_costs_ind_projection_sources.xlsx",  
                        add_sources=add_sources)

### Fixed costs

In [ ]:
fixed_costs_ind_projection_dict, fixed_costs_ind_projection_df = extract_stats(fixed_costs_ind_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(fixed_costs_ind_df, years_dict, highest_potential_ind_cols, 
                          title="Fixe Kosten für ausgewählte Industrieprozesse - Projektion", 
                          ylabel="fixe Kosten in €_2018/kW*a",  
                          colors=colors,
                          ylim=[0, 3000],
                          use_colors=use_colors,
                          use_limits=use_limits, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="fixed_costs_ind")

In [ ]:
if write_sources:
    save_future_sources(fixed_costs_ind_df, highest_potential_ind_cols, path_folder=path_folder_sources, filename="fixed_costs_ind_projection_sources.xlsx",  
                        add_sources=add_sources)

## Trade, commerce and services

### Select some processes for further analysis
Method and criterions used for selection:
* Select those with highest demand response potential in the status quo.
* Select those with highest demand response potential in the 2050.
* Combine the two, i.e. remove those that only show large potentials in one of the categories.
* Analysis is separately carried out for positive and negative potentials to select the top 5 for either category.
* For the time- and cost-related parameters, the top 3 of every category is chosen.

In [ ]:
# Get the 5 processes with the largest positive potential for the status quo and 2050
high_potential_pos_tcs_cols_SQ = get_nlargest(potential_pos_tcs_ave_years_dict, year="SQ", metric=metric, n=15)
high_potential_pos_tcs_cols_2050 = get_nlargest(potential_pos_tcs_ave_years_dict, year="2050", metric=metric, n=15)

# Get the 5 processes with the largest negative potential for the status quo and 2050
high_potential_neg_tcs_cols_SQ = get_nlargest(potential_neg_tcs_ave_years_dict, year="SQ", metric=metric, n=6)
high_potential_neg_tcs_cols_2050 = get_nlargest(potential_neg_tcs_ave_years_dict, year="2050", metric=metric, n=6)

# Select the processes with the largest potential by combining the two
high_potential_pos_tcs_cols = list(set(high_potential_pos_tcs_cols_SQ).intersection(set(high_potential_pos_tcs_cols_2050)))
high_potential_neg_tcs_cols = list(set(high_potential_neg_tcs_cols_SQ).intersection(set(high_potential_neg_tcs_cols_2050)))

print(high_potential_pos_tcs_cols)
print(high_potential_neg_tcs_cols)

In [ ]:
# Get the 3 processes with the largest positive potential for the status quo and 2050
highest_potential_pos_tcs_cols_SQ = get_nlargest(potential_pos_tcs_ave_years_dict, year="SQ", metric=metric, n=5)
highest_potential_pos_tcs_cols_2050 = get_nlargest(potential_pos_tcs_ave_years_dict, year="2050", metric=metric, n=5)

# Get the 3 processes with the largest negative potential for the status quo and 2050
highest_potential_neg_tcs_cols_SQ = get_nlargest(potential_neg_tcs_ave_years_dict, year="SQ", metric=metric, n=5)
highest_potential_neg_tcs_cols_2050 = get_nlargest(potential_neg_tcs_ave_years_dict, year="2050", metric=metric, n=5)

# Select the processes with the largest potential by combining the two
highest_potential_tcs_cols = list(set(highest_potential_pos_tcs_cols_SQ).intersection(set(highest_potential_pos_tcs_cols_2050)))
highest_potential_neg_tcs_cols = list(set(highest_potential_neg_tcs_cols_SQ).intersection(set(highest_potential_neg_tcs_cols_2050)))
highest_potential_tcs_cols.extend(highest_potential_neg_tcs_cols)
highest_potential_tcs_cols = list(set(highest_potential_tcs_cols))

print(highest_potential_tcs_cols)

Alternative approach: No systematic choice of processes, but selection of those of interest.

In [ ]:
if not use_nlargest_cols:
    if not modify_categories:
        tcs_cols = ["Prozesskälte gesamt", "Belüftung gesamt", "Klimaanlagen", "Trinkwasserpumpen"]
    elif aggregation_mode == "granular":
        tcs_cols = ["Prozesskälte", "Belüftung", "Klimakälte", "Pumpenanwendungen in der Wasserversorgung"]
    elif aggregation_mode == "broad":
        tcs_cols = ["Prozesskälte", "Belüftung", "Klimakälte", "Pumpenanwendungen in der Wasserversorgung"]
    high_potential_pos_tcs_cols = tcs_cols
    high_potential_neg_tcs_cols = tcs_cols
    highest_potential_tcs_cols = tcs_cols
    print(tcs_cols)

### Positive load shift potential

Create a dict containing the statistical info per year as well as a DataFrame containing the info on the development of the respective parameter

In [ ]:
potential_pos_tcs_ave_projection_dict, potential_pos_tcs_ave_projection_df = extract_stats(potential_pos_tcs_ave_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(potential_pos_tcs_ave_df, years_dict, high_potential_pos_tcs_cols, 
                          title="Positives Lastmanagementpotenzial (Lastreduktion) für ausgewählte Anwendungen im GHD-Sektor - Projektion",
                          ylabel="Potenzial in MW", 
                          colors=colors,
                          ylim=[0, 1000],
                          use_colors=use_colors,
                          use_limits=False,
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="potential_pos_tcs_ave")

In [ ]:
if write_sources:
    save_future_sources(potential_pos_tcs_ave_df, high_potential_pos_tcs_cols, path_folder=path_folder_sources, filename="potential_pos_tcs_ave_projection_sources.xlsx",  
                        add_sources=add_sources)

### Negative load shift potential

In [ ]:
potential_neg_tcs_ave_projection_dict, potential_neg_tcs_ave_projection_df = extract_stats(potential_neg_tcs_ave_years_dict, years_dict)

In [ ]:
create_projection_boxplot(potential_neg_tcs_ave_df, years_dict, high_potential_neg_tcs_cols, 
                          title="Negatives Lastmanagementpotenzial (Lasterhöhung) für ausgewählte Anwendungen im GHD-Sektor - Projektion",
                          ylabel="Potenzial in MW",  
                          colors=colors,
                          ylim=[-3000, 0],
                          use_colors=use_colors,
                          negate=True,
                          use_limits=False, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="potential_neg_tcs_ave")

In [ ]:
if write_sources:
    save_future_sources(potential_neg_tcs_ave_df, high_potential_neg_tcs_cols, path_folder=path_folder_sources, filename="potential_neg_tcs_ave_projection_sources.xlsx",  
                        add_sources=add_sources)

Visualize the data using a grouped boxplot

### Specific investments

In [ ]:
specific_investments_tcs_projection_dict, specific_investments_tcs_projection_df = extract_stats(specific_investments_tcs_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(specific_investments_tcs_df, years_dict, highest_potential_tcs_cols, 
                          title="Spezifische Investitionsausgaben für ausgewählte Anwendungen im GHD-Sektor - Projektion",
                          ylabel="spezifische Investitionsausgaben in €_2018/kW", 
                          colors=colors,
                          ylim=[0, 3000],
                          use_colors=use_colors,
                          use_limits=use_limits, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="specific_investments_tcs")

In [ ]:
if write_sources:
    save_future_sources(specific_investments_tcs_df, highest_potential_tcs_cols, path_folder=path_folder_sources, filename="specific_investments_tcs_projection_sources.xlsx",  
                        add_sources=add_sources)

### Variable costs

In [ ]:
variable_costs_tcs_projection_dict, variable_costs_tcs_projection_df = extract_stats(variable_costs_tcs_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(variable_costs_tcs_df, years_dict, highest_potential_tcs_cols, 
                          title="Variable Kosten für ausgewählte Anwendungen im GHD-Sektor - Projektion", 
                          ylabel="variable Kosten in €_2018/MWh",
                          colors=colors,
                          ylim=[0, 3000],
                          use_colors=use_colors,
                          use_limits=use_limits, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="variable_costs_tcs")

In [ ]:
if write_sources:
    save_future_sources(variable_costs_tcs_df, highest_potential_tcs_cols, path_folder=path_folder_sources, filename="variable_costs_tcs_projection_sources.xlsx",  
                        add_sources=add_sources)

### Fixed costs

In [ ]:
fixed_costs_tcs_projection_dict, fixed_costs_tcs_projection_df = extract_stats(fixed_costs_tcs_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(fixed_costs_tcs_df, years_dict, highest_potential_tcs_cols, 
                          title="Fixe Kosten für ausgewählte Anwendungen im GHD-Sektor - Projektion", 
                          ylabel="fixe Kosten in €_2018/kW*a",
                          colors=colors,
                          ylim=[0, 3000],
                          use_colors=use_colors,
                          use_limits=use_limits, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="fixed_costs_tcs")

In [ ]:
if write_sources:
    save_future_sources(fixed_costs_tcs_df, highest_potential_tcs_cols, path_folder=path_folder_sources, filename="fixed_costs_tcs_projection_sources.xlsx",  
                        add_sources=add_sources)

## Households

### Select some processes for further analysis
Method and criterions used for selection:
* Select those with highest demand response potential in the status quo.
* Select those with highest demand response potential in the 2050.
* Combine the two, i.e. remove those that only show large potentials in one of the categories.
* Analysis is separately carried out for positive and negative potentials to select the top 5 for either category.
* For the time- and cost-related parameters, the top 3 of every category is chosen.

In [ ]:
# Get the 5 processes with the largest positive potential for the status quo and 2050
high_potential_pos_hoho_cols_SQ = get_nlargest(potential_pos_hoho_ave_years_dict, year="SQ", metric=metric, n=n)
high_potential_pos_hoho_cols_2050 = get_nlargest(potential_pos_hoho_ave_years_dict, year="2050", metric=metric, n=n)

# Get the 5 processes with the largest negative potential for the status quo and 2050
high_potential_neg_hoho_cols_SQ = get_nlargest(potential_neg_hoho_ave_years_dict, year="SQ", metric=metric, n=n)
high_potential_neg_hoho_cols_2050 = get_nlargest(potential_neg_hoho_ave_years_dict, year="2050", metric=metric, n=n)

# Select the processes with the largest potential by combining the two
high_potential_pos_hoho_cols = list(set(high_potential_pos_hoho_cols_SQ).intersection(set(high_potential_pos_hoho_cols_2050)))
high_potential_neg_hoho_cols = list(set(high_potential_neg_hoho_cols_SQ).intersection(set(high_potential_neg_hoho_cols_2050)))

print(high_potential_pos_hoho_cols)
print(high_potential_neg_hoho_cols)

In [ ]:
# Get the 3 processes with the largest positive potential for the status quo and 2050
highest_potential_pos_hoho_cols_SQ = get_nlargest(potential_pos_hoho_ave_years_dict, year="SQ", metric=metric, n=3)
highest_potential_pos_hoho_cols_2050 = get_nlargest(potential_pos_hoho_ave_years_dict, year="2050", metric=metric, n=3)

# Get the 3 processes with the largest negative potential for the status quo and 2050
highest_potential_neg_hoho_cols_SQ = get_nlargest(potential_neg_hoho_ave_years_dict, year="SQ", metric=metric, n=3)
highest_potential_neg_hoho_cols_2050 = get_nlargest(potential_neg_hoho_ave_years_dict, year="2050", metric=metric, n=3)

# Select the processes with the largest potential by combining the two
highest_potential_hoho_cols = list(set(highest_potential_pos_hoho_cols_SQ).intersection(set(highest_potential_pos_hoho_cols_2050)))
highest_potential_neg_hoho_cols = list(set(highest_potential_neg_hoho_cols_SQ).intersection(set(highest_potential_neg_hoho_cols_2050)))
highest_potential_hoho_cols.extend(highest_potential_neg_hoho_cols)
highest_potential_hoho_cols = list(set(highest_potential_hoho_cols))

print(highest_potential_hoho_cols)

In [ ]:
if not use_nlargest_cols:
    if not modify_categories:
        hoho_cols = ["Wärmepumpen", "Nachtspeicher Haushalte", "Klimaanlagen", "Kühlschrank"]
    elif aggregation_mode == "granular":
        hoho_cols = ["Wärmepumpen", "Nachtspeicherheizungen", "Klimakälte", "Kühlschränke"]
    elif aggregation_mode == "broad":
        hoho_cols = ["Wärmepumpen", "Nachtspeicherheizungen", "Klimakälte", "Kühl- und Gefrieranwendungen"]
    high_potential_pos_hoho_cols = hoho_cols
    high_potential_neg_hoho_cols = hoho_cols
    highest_potential_hoho_cols = hoho_cols
    print(hoho_cols)

### Positive load shift potential

Create a dict containing the statistical info per year as well as a DataFrame containing the info on the development of the respective parameter

In [ ]:
potential_pos_hoho_ave_projection_dict, potential_pos_hoho_ave_projection_df = extract_stats(potential_pos_hoho_ave_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(potential_pos_hoho_ave_df, years_dict, high_potential_pos_hoho_cols, 
                          title="Projektion des positiven Lastmanagementpotenzials (Lastreduktion) für ausgewählte Haushaltsanwendungen",
                          ylabel="durchn. positives Potenzial in MW",
                          colors=colors,
                          ylim=[0, 12000],
                          use_colors=use_colors,
                          use_limits=False, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="potential_pos_hoho_ave")

In [ ]:
if write_sources:
    save_future_sources(potential_pos_hoho_ave_df, high_potential_pos_hoho_cols, path_folder=path_folder_sources, filename="potential_pos_hoho_ave_projection_sources.xlsx",  
                        add_sources=add_sources)

### Negative load shift potential

In [ ]:
potential_neg_hoho_ave_projection_dict, potential_neg_hoho_ave_projection_df = extract_stats(potential_neg_hoho_ave_years_dict, years_dict)

In [ ]:
create_projection_boxplot(potential_neg_hoho_ave_df, years_dict, high_potential_neg_hoho_cols, 
                          title="Negatives Lastmanagementpotenzial (Lasterhöhung) für ausgewählte Haushaltsanwendungen - Projektion",
                          ylabel="Potenzial in MW", 
                          colors=colors,
                          ylim=[-12000, 0],
                          use_colors=use_colors,
                          negate=True,
                          use_limits=False, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="potential_neg_hoho_ave")

In [ ]:
if write_sources:
    save_future_sources(potential_neg_hoho_ave_df, high_potential_neg_hoho_cols, path_folder=path_folder_sources, filename="potential_neg_hoho_ave_projection_sources.xlsx",  
                        add_sources=add_sources)

Visualize the data using a grouped boxplot

### Specific investments

In [ ]:
specific_investments_hoho_projection_dict, specific_investments_hoho_projection_df = extract_stats(specific_investments_hoho_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(specific_investments_hoho_df, years_dict, highest_potential_hoho_cols, 
                          title="Spezifische Investitionsausgaben für ausgewählte Haushaltsanwendungen - Projektion",
                          ylabel="spezifische Investitionsausgaben in €_2018/kW",
                          colors=colors,
                          ylim=[0, 1500],
                          use_colors=use_colors,
                          use_limits=False, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="specific_investments_hoho")

In [ ]:
if write_sources:
    save_future_sources(specific_investments_hoho_df, highest_potential_hoho_cols, path_folder=path_folder_sources, filename="specific_investments_hoho_projection_sources.xlsx",  
                        add_sources=add_sources)

### Variable costs

In [ ]:
variable_costs_hoho_projection_dict, variable_costs_hoho_projection_df = extract_stats(variable_costs_hoho_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(variable_costs_hoho_df, years_dict, highest_potential_hoho_cols, 
                          title="Variable Kosten für ausgewählte Haushaltsanwendungen - Projektion", 
                          ylabel="variable Kosten in €_2018/MWh",
                          colors=colors,
                          ylim=[0, 3000],
                          use_colors=use_colors,
                          use_limits=use_limits, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="variable_costs_hoho")

In [ ]:
if write_sources:
    save_future_sources(variable_costs_hoho_df, highest_potential_hoho_cols, path_folder=path_folder_sources, filename="variable_costs_hoho_projection_sources.xlsx",  
                        add_sources=add_sources)

### Fixed costs

In [ ]:
fixed_costs_hoho_projection_dict, fixed_costs_hoho_projection_df = extract_stats(fixed_costs_hoho_years_dict, years_dict)

Visualize the data using a grouped boxplot

In [ ]:
create_projection_boxplot(fixed_costs_hoho_df, years_dict, highest_potential_hoho_cols, 
                          title="Fixe Kosten für ausgewählte Anwendungen im GHD-Sektor - Projektion", 
                          ylabel="fixe Kosten in €_2018/kW*a", 
                          colors=colors,
                          ylim=[0, 3000],
                          use_colors=use_colors,
                          use_limits=use_limits, 
                          sample_stats=sample_stats, 
                          add_sources=add_sources, 
                          swarmplot=swarmplot, 
                          savefig=savefig, 
                          filename="fixed_costs_hoho")

In [ ]:
if write_sources:
    save_future_sources(fixed_costs_hoho_df, highest_potential_hoho_cols, path_folder=path_folder_sources, filename="fixed_costs_hoho_projection_sources.xlsx",  
                        add_sources=add_sources)